---
# **Dynamic Shrinkage Method for Hierarchical Risk Parity to Improve Portfolio Stability and Performance**

---

Student Name: Xinyi (Cynthia) Shen



# Preliminaries

## Libraries

In [1]:
# Import
!pip install plotly
!pip install statsmodels matplotlib
!pip install arch
!pip install -U kaleido

# path
from google.colab import drive
import os
# data
import numpy as np
import pandas as pd
# plot
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# stats
from statsmodels.tsa.stattools import adfuller
# scipy
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform
from scipy.optimize import minimize
from scipy.stats import multivariate_normal
# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import LedoitWolf
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
# arch
from arch import arch_model
# joblib
from joblib import Parallel, delayed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.1/985.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 27.1 MB/s eta 0:00:00


## Input Data

In [2]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the path to the notebook's directory on Google Drive
notebook_directory = "/content/drive/MyDrive/data"

# Construct the path to the CSV file
DEFITVL_path = os.path.join(notebook_directory, "chain-dataset-All.csv") # DeFi Total Value Locked (TVL) Index (ticker: DEFITVL Index)
DEFITVL_df = pd.read_csv(DEFITVL_path)
BCOM_path = os.path.join(notebook_directory, "BCOM.csv") # Bloomberg Commodities Index (ticker: BCOM Index)
BCOM_df = pd.read_csv(BCOM_path)

# Unique and less-studied cryptocurrencies
crypto_paths = [
    os.path.join(notebook_directory, "ftm-usd-max.csv"), # Fantom (ticker: FTM)
    os.path.join(notebook_directory, "rune-usd-max.csv"), # ThorChain (ticker: RUNE)
    os.path.join(notebook_directory, "crv-usd-max.csv"), # Curve DAO (ticker: CRV)
    os.path.join(notebook_directory, "sushi-usd-max.csv"), # Sushi (ticker: sushi)
    os.path.join(notebook_directory, "yfi-usd-max.csv") # Yearn.Finance (ticker: YFI)
]

# Sensitivity analysis cryptocurrencies
bitcoin_paths = [
    os.path.join(notebook_directory, 'bch-usd-max.csv'), # Bitcoin Cash (BCH)
    os.path.join(notebook_directory, "bsv-usd-max.csv"), # Bitcoin SV (BSV)
    os.path.join(notebook_directory, "ltc-usd-max.csv"), # Litecoin (LTC)
    os.path.join(notebook_directory, "btg-usd-max.csv"), # Bitcoin Gold (BTG)
    os.path.join(notebook_directory, "bcd-usd-max.csv"), # Bitcoin Diamond (BCD)
    os.path.join(notebook_directory, "wbtc-usd-max.csv"), # Wrapped Bitcoin (WBTC)
    os.path.join(notebook_directory, "xec-usd-max.csv"), # eCash (XEC)
    os.path.join(notebook_directory, "nmr-usd-max.csv"), # Namecoin (NMC)
    os.path.join(notebook_directory, "rvn-usd-max.csv") # Ravencoin (RVN)
]

# Macroeconomic variables
rf_rate_path = os.path.join(notebook_directory, "USGG5YR.csv") # U.S. 5-Year Treasury Yield (ticker: USGG5YR Index)

Mounted at /content/drive


# Data Management

In [3]:
def prepare_logreturns_and_rf_rate(crypto_paths, rf_rate_path, start_date, end_date):
    """
    Prepares the log returns dataset and reindexed risk-free rate dataset.

    Parameters:
    - crypto_paths: List of file paths for each cryptocurrency dataset.
    - rf_rate_path: Path for the risk-free rate dataset.
    - start_date: Start date for the analysis period (format: "YYYY-MM-DD").
    - end_date: End date for the analysis period (format: "YYYY-MM-DD").

    Returns:
    - log_returns_df: DataFrame containing log returns for each cryptocurrency.
    - rf_rate_df: DataFrame of the reindexed risk-free rate for the specified period.
    """

    # Function to preprocess each cryptocurrency DataFrame
    def preprocess_crypto_data(file_path):
        df = pd.read_csv(file_path)
        df['Date'] = pd.to_datetime(df['snapped_at']).dt.date
        df = df.rename(columns={'price': 'Price', 'market_cap': 'Market Capitalization', 'total_volume': 'Total Volume'})
        df = df[['Date', 'Price', 'Market Capitalization', 'Total Volume']]
        df.set_index('Date', inplace=True)
        df = df.sort_index()
        df['Log Returns'] = np.log(df['Price'] / df['Price'].shift(1))
        df = df.dropna(subset=['Log Returns'])
        return df['Log Returns']  # Return only log returns column

    # Process each cryptocurrency dataset
    crypto_log_returns = {}
    for path in crypto_paths:
        crypto_name = os.path.basename(path).split('-')[0].upper()  # Extract name from filename
        crypto_log_returns[crypto_name] = preprocess_crypto_data(path)

    # Create combined log returns DataFrame
    log_returns_df = pd.DataFrame(crypto_log_returns)

    # Load and preprocess the risk-free rate data
    rf_rate_df = pd.read_csv(rf_rate_path)
    rf_rate_df['Date'] = pd.to_datetime(rf_rate_df['Date']).dt.date
    rf_rate_df.set_index('Date', inplace=True)

    # Create date range and reindex dataframes
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    log_returns_df = log_returns_df.reindex(date_range)
    rf_rate_df = rf_rate_df.reindex(date_range).ffill().bfill()  # Forward and backfill missing values

    return log_returns_df, rf_rate_df

In [4]:
# In-sample, out-of-sample, sensitivity log return and risk-free rate data

# In-sample
start_date = "2020-10-01"; end_date = "2021-09-30"
log_returns_20to21, rf_rate_20to21 = prepare_logreturns_and_rf_rate(crypto_paths, rf_rate_path, start_date, end_date)

# Out-of-sample
start_date = "2021-10-01"; end_date = "2022-09-30"

log_returns_21to22, rf_rate_21to22 = prepare_logreturns_and_rf_rate(crypto_paths, rf_rate_path, start_date, end_date)

start_date = "2022-10-01"; end_date = "2023-09-30"
log_returns_22to23, rf_rate_22to23 = prepare_logreturns_and_rf_rate(crypto_paths, rf_rate_path, start_date, end_date)

start_date = "2023-10-01"; end_date = "2024-09-30"
log_returns_23to24, rf_rate_23to24 = prepare_logreturns_and_rf_rate(crypto_paths, rf_rate_path, start_date, end_date)

# Bitcoin
start_date = "2023-10-01"; end_date = "2024-09-30"
log_returns_bit23to24, rf_rate_23to24 = prepare_logreturns_and_rf_rate(bitcoin_paths, rf_rate_path, start_date, end_date)

# In-sample + Out-of-sample (4 years)
start_date = "2020-10-01"; end_date = "2024-09-30"
log_returns_20to24, rf_rate_20to24 = prepare_logreturns_and_rf_rate(crypto_paths, rf_rate_path, start_date, end_date)

# Methodology

## Original Methodology

### Dynamic Shrinkage Hierachical Risk Parity (DS-HRP) Portfolio Weights

In [5]:
def dshrp_weights(returns, rf_rate, t, alpha, kappa_0):
    """Calculate portfolio weights using DSHRP method."""
    # Step 1: Hierarchical Clustering
    corr_matrix = returns.corr()
    distance_matrix = np.sqrt(0.5 * (1 - corr_matrix))
    condensed_distance_matrix = squareform(distance_matrix)
    link = linkage(condensed_distance_matrix, method='complete')
    cluster_labels = fcluster(link, t, criterion='distance')

    # Step 2: Dynamic Shrinkage for Clusters
    shrunk_cov_matrices = {}
    for label in np.unique(cluster_labels):
        assets_in_cluster = returns.columns[cluster_labels == label]
        cluster_returns = returns[assets_in_cluster]

        # Ledoit-Wolf shrinkage
        lw = LedoitWolf()
        cov_matrix = lw.fit(cluster_returns).covariance_

        # Compute shrinkage factor dynamically
        eigenvalues = np.linalg.eigvals(cov_matrix)
        condition_number = np.max(eigenvalues) / np.min(eigenvalues)
        shrinkage_factor = 1 / (1 + np.exp(-alpha * (condition_number - kappa_0)))

        # Shrinkage-adjusted covariance matrix
        shrunk_cov = (1 - shrinkage_factor) * cov_matrix + shrinkage_factor * np.eye(len(cov_matrix))
        shrunk_cov_matrices[label] = shrunk_cov

    # Step 3: Quasi-Diagonalization of the Covariance Matrix
    def pad_matrix(matrix, target_size):
        padded = np.zeros((target_size, target_size))
        padded[:matrix.shape[0], :matrix.shape[1]] = matrix
        return padded

    max_size = max(matrix.shape[0] for matrix in shrunk_cov_matrices.values())
    padded_cov_matrices = [pad_matrix(shrunk_cov_matrices[i], max_size) for i in shrunk_cov_matrices]

    combined_cov_matrix = np.block([
        [padded_cov_matrices[i] if i == j else np.zeros((max_size, max_size))
         for j in range(len(padded_cov_matrices))]
        for i in range(len(padded_cov_matrices))
    ])

    sorted_indices = np.argsort(cluster_labels)
    quasi_diag_cov = combined_cov_matrix[sorted_indices, :][:, sorted_indices]

    # Step 4: Recursive Bisection for Weights Calculation
    std_dev = np.sqrt(np.diag(quasi_diag_cov))
    inv_vol = np.zeros_like(std_dev)
    non_zero_std = std_dev > 0
    inv_vol[non_zero_std] = 1 / std_dev[non_zero_std]

    if inv_vol.sum() > 0:
        weights = inv_vol / inv_vol.sum()
    else:
        weights = np.zeros_like(inv_vol)

    # Step 5: Calculate Performance Metrics with returns and rf_rates
    SR, vol, MDD = performance(weights, returns, rf_rate)

    return weights, SR, vol, MDD

### Traditional Hierachical Risk Parity (HRP) Portfolio Weights

---



In [6]:
def hrp_weights(returns):
    """
    Calculate the Hierarchical Risk Parity (HRP) portfolio weights.

    Parameters:
    - returns: DataFrame of asset returns.

    Returns:
    - hrp_weights: Array of HRP portfolio weights for each asset.
    """

    # Step 1: Calculate pairwise correlation and distance matrices
    corr_matrix = returns.corr()
    distance_matrix = np.sqrt(0.5 * (1 - corr_matrix))

    # Convert to condensed form for hierarchical clustering
    condensed_distance_matrix = squareform(distance_matrix, checks=False)  # Convert to condensed format
    link = linkage(condensed_distance_matrix, method='single')  # 'single' linkage clustering

    # HRP: Calculate cluster variance (Vectorized)
    def hrp_get_cluster_var(cov, cluster_indices):
        """Calculate variance of the cluster using a vectorized approach."""
        sub_cov = cov[np.ix_(cluster_indices, cluster_indices)]
        w = np.full(sub_cov.shape[0], 1 / sub_cov.shape[0])
        return w @ sub_cov @ w  # Matrix multiplication

    # HRP: Recursive bisection for portfolio weights (Iterative implementation)
    def hrp_iterative_bisection(cov, link):
        """Iteratively allocate weights based on risk contribution."""
        clusters = fcluster(link, t=1.5, criterion='maxclust')
        weights = np.zeros(len(cov))

        for cluster_id in np.unique(clusters):
            cluster_indices = np.where(clusters == cluster_id)[0]
            if len(cluster_indices) == 1:
                weights[cluster_indices[0]] = 1.0  # Single asset
                continue

            # Calculate cluster variances
            var_left = hrp_get_cluster_var(cov, cluster_indices[: len(cluster_indices) // 2])
            var_right = hrp_get_cluster_var(cov, cluster_indices[len(cluster_indices) // 2:])

            # Allocate capital based on inverse variance
            alloc_left = var_right / (var_left + var_right)
            alloc_right = var_left / (var_left + var_right)

            # Distribute weights across left and right clusters
            weights[cluster_indices[: len(cluster_indices) // 2]] = alloc_left
            weights[cluster_indices[len(cluster_indices) // 2:]] = alloc_right

        return weights / weights.sum()  # Normalize weights

    # Step 3: Compute the covariance matrix of returns
    cov_matrix = returns.cov().values

    # Step 4: Apply iterative bisection to compute weights
    hrp_weights = hrp_iterative_bisection(cov_matrix, link)

    return hrp_weights

### Markowitz's Mean-Variance Portfolios Weights

In [7]:
def mv_weights(returns):
    """
    Calculate the Markowitz minimum volatility portfolio weights.

    Parameters:
    - returns: DataFrame of asset returns.

    Returns:
    - mv_weights: Array of minimum volatility portfolio weights for each asset.
    """

    # Step 1: Calculate expected returns and covariance matrix
    expected_returns = returns.mean() * 252  # Annualized
    cov_matrix = returns.cov() * 252  # Annualized

    # Step 2: Define the objective function for portfolio optimization
    def portfolio_volatility(weights, cov_matrix):
        """Calculate the portfolio volatility."""
        return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Step 3: Constraints (weights sum to 1)
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

    # Step 4: Bounds for weights (0 to 1 for long-only portfolio)
    bounds = tuple((0, 1) for _ in range(len(expected_returns)))

    # Step 5: Initial guess (equal weighting)
    initial_weights = np.ones(len(expected_returns)) / len(expected_returns)

    # Step 6: Optimize for minimum volatility
    result = minimize(portfolio_volatility, initial_weights,
                      args=(cov_matrix,), method='SLSQP',
                      bounds=bounds, constraints=constraints)

    # Get the optimal weights
    mv_weights = result.x

    return mv_weights

### Performance Evaluation

In [8]:
def performance(portfolio_weights, returns, rf_rate):
    """Compute Sharpe Ratio, Volatility, and MDD."""
    sharpe_ratios_df = pd.DataFrame()
    sharpe_ratios_df.index = returns.index
    sharpe_ratios_df["Weighted Average Return"] = (returns * portfolio_weights).sum(axis=1)

    # Ensure rf_rate is a single column series and align with returns index
    if isinstance(rf_rate, pd.DataFrame) and rf_rate.shape[1] > 1:
        rf_rate = rf_rate.iloc[:, 0]  # Take the first column
    rf_rate = rf_rate.reindex(returns.index).ffill()  # Ensure alignment

    sharpe_ratios_df["Adjusted Risk-free Rate"] = (1 + rf_rate / 100) ** (1 / 252) - 1

    # Row-wise volatility
    def calculate_row_volatility(row, weights):
        cov_matrix = np.diag(row.values ** 2)
        portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
        return np.sqrt(portfolio_variance)

    volatilities = returns.apply(lambda row: calculate_row_volatility(row, portfolio_weights), axis=1)
    sharpe_ratios_df["Portfolio Volatility"] = volatilities

    sharpe_ratios_df["Sharpe Ratio"] = (
        sharpe_ratios_df["Weighted Average Return"] - sharpe_ratios_df["Adjusted Risk-free Rate"]
    ) / sharpe_ratios_df["Portfolio Volatility"]

    # Maximum Drawdown
    cumulative_returns = (1 + sharpe_ratios_df["Weighted Average Return"]).cumprod()
    running_max = cumulative_returns.cummax()
    max_drawdown = ((running_max - cumulative_returns) / running_max).max()

    SR = sharpe_ratios_df["Sharpe Ratio"].mean()
    vol = volatilities.mean()
    MDD = max_drawdown

    return SR, vol, MDD

### Find Optimal DS-HRP Parameters for Best Sharpe Ratio

In [9]:
def find_optimal_params(returns, rf_rates, param_grid):
    """Find the optimal t, alpha, and kappa_0 for maximum SR."""
    best_sr, best_params = -np.inf, None
    best_weights, best_vol, best_mdd = None, None, None

    # Iterate over all combinations in the parameter grid
    for params in ParameterGrid(param_grid):
        t, alpha, kappa_0 = params['t'], params['alpha'], params['kappa_0']

        # Get portfolio weights and performance metrics using current parameters
        weights, sr, vol, mdd = dshrp_weights(returns, rf_rates, t, alpha, kappa_0)

        # Update if current Sharpe Ratio is better than the previous best
        if sr > best_sr:
            best_sr = sr
            best_params = params
            best_weights = weights
            best_vol = vol
            best_mdd = mdd

    # Display the optimal parameters and corresponding metrics
    print("\nOptimal DSHRP Parameters:")
    print(f"t: {best_params['t']}, alpha: {best_params['alpha']}, kappa_0: {best_params['kappa_0']}")
    print(f"Optimal Portfolio Weights: {best_weights}")
    print(f"Optimal Sharpe Ratio: {best_sr:.2%}")
    print(f"Portfolio Volatility: {best_vol:.2%}")
    print(f"Maximum Drawdown: {best_mdd:.2%}")

    return best_params, best_weights, best_sr, best_vol, best_mdd

### Define DS-HRP Parameter Grid

In [10]:
# Define the parameter grid
param_grid_dshrp = {
    't': np.arange(0.1, 1.0, 0.05),
    'alpha': np.arange(0.1, 2.1, 0.1),
    'kappa_0': np.arange(10, 50, 1)
}

## Further Enhancement Methodology

### Higher Order DS-HRP Portfolio Weights

In [11]:
def higher_order_dshrp_weights(data, rf_rate, alpha, beta, baseline_volatility, baseline_correlation, gamma, delta):
    """
    Calculate dynamically adjusted covariance matrices, optimize weights for Sharpe ratio,
    and use recursive bisection for weights calculation.

    Parameters:
    - data: A Pandas DataFrame with rows as time (t) and columns as asset returns.
    - rf_rate: Risk-free rate for Sharpe ratio calculation.
    - alpha: Scaling factor for skewness adjustment.
    - beta: Scaling factor for kurtosis adjustment.
    - baseline_volatility: Baseline volatility (sigma_0).
    - baseline_correlation: Baseline correlation threshold (rho_0).
    - gamma: Sensitivity to volatility.
    - delta: Sensitivity to correlation.

    Returns:
    - final_weights: Final portfolio weights at the last time step.
    - SR: Sharpe Ratio at the last time step.
    - vol: Portfolio volatility at the last time step.
    - MDD: Maximum drawdown at the last time step.
    """
    n_assets = data.shape[1]
    adjusted_covariances = []
    final_weights = None

    # Calculate expected returns as average log returns
    expected_returns = data.mean().values

    # Iterate over rows (each time t)
    for t in range(len(data)):
        row_data = data.iloc[:t + 1]

        # Skip rows where covariance computation is invalid
        if len(row_data) < 2:
            # Not enough data for covariance calculation
            continue

        centered_data = row_data - row_data.mean()
        cov_matrix = row_data.cov().values

        # Skewness tensor
        skewness_tensor = np.einsum('ti,tj,tk->ijk', centered_data.values, centered_data.values, centered_data.values) / len(row_data)

        # Kurtosis tensor
        kurtosis_tensor = np.einsum('ti,tj,tk,tl->ijkl', centered_data.values, centered_data.values, centered_data.values, centered_data.values) / len(row_data)
        kurtosis_tensor -= 3 * np.einsum('ij,kl->ijkl', cov_matrix, cov_matrix)

        # Portfolio volatility and average correlation
        initial_weights = np.ones(n_assets) / n_assets
        portfolio_variance = np.dot(initial_weights.T, np.dot(cov_matrix, initial_weights))
        portfolio_volatility = np.sqrt(portfolio_variance)
        correlation_matrix = row_data.corr().values
        upper_triangle_indices = np.triu_indices_from(correlation_matrix, k=1)
        avg_correlation = np.nanmean(correlation_matrix[upper_triangle_indices])  # Avoid issues with NaN correlations

        # Dynamic shrinkage factor lambda
        lambda_dynamic = 1 / (1 + np.exp(-gamma * (portfolio_volatility - baseline_volatility) + delta * (avg_correlation - baseline_correlation)))

        # Adjusted covariance matrix
        nonlin_covariance = cov_matrix + alpha * np.sum(skewness_tensor, axis=2) + beta * np.sum(kurtosis_tensor, axis=(2, 3))
        adjusted_covariance = (1 - lambda_dynamic) * cov_matrix + lambda_dynamic * nonlin_covariance

        # Ensure covariance matrix is symmetric
        adjusted_covariance = (adjusted_covariance + adjusted_covariance.T) / 2

        # Ensure positive semi-definiteness of covariance matrix
        eigvals, eigvecs = np.linalg.eigh(adjusted_covariance)
        eigvals = np.clip(eigvals, 1e-8, None)  # Replace negative eigenvalues with a small positive value
        adjusted_covariance = eigvecs @ np.diag(eigvals) @ eigvecs.T

        adjusted_covariances.append(adjusted_covariance)

        # Recursive Bisection for Weights Calculation
        std_dev = np.sqrt(np.diag(adjusted_covariance))
        inv_vol = np.zeros_like(std_dev)
        non_zero_std = std_dev > 0
        inv_vol[non_zero_std] = 1 / std_dev[non_zero_std]

        if inv_vol.sum() > 0:
            weights = inv_vol / inv_vol.sum()
        else:
            weights = np.zeros_like(inv_vol)

        # Store the final weights
        final_weights = weights

    # Compute performance metrics using final weights
    SR, vol, MDD = performance(final_weights, data, rf_rate)

    return final_weights, SR, vol, MDD

### Find Optimal Higher Order DS-HRP Parameters for Best Sharpe Ratio

In [12]:
def find_optimal_params_higher_order(returns, rf_rate, param_grid, file_name, n_jobs=-1):
    """
    Find the optimal parameters (alpha, beta, baseline_volatility, baseline_correlation, gamma, delta)
    to maximize the Sharpe ratio using the higher-order DSHRP method, and plot Sharpe ratio improvement.

    Parameters:
    - returns: A Pandas DataFrame of asset returns.
    - rf_rates: A vector of risk-free rates corresponding to the returns.
    - param_grid: A dictionary defining the parameter grid for alpha, beta, baseline_volatility,
                  baseline_correlation, gamma, and delta.
    - n_jobs: Number of parallel jobs for parameter search (-1 uses all CPUs).

    Returns:
    - best_params: The optimal parameters for maximum Sharpe ratio.
    - best_weights: Optimal portfolio weights using the best parameters.
    - best_sr: Sharpe ratio for the optimal portfolio.
    - best_vol: Portfolio volatility for the optimal portfolio.
    - best_mdd: Maximum drawdown for the optimal portfolio.
    """

    def evaluate_params(idx, params):
        """Evaluate Sharpe ratio for given parameters."""
        alpha = params['alpha']
        beta = params['beta']
        baseline_volatility = params['baseline_volatility']
        baseline_correlation = params['baseline_correlation']
        gamma = params['gamma']
        delta = params['delta']

        # Calculate weights and performance metrics using current parameters
        weights, sr, vol, mdd = higher_order_dshrp_weights(
            returns,
            rf_rate,
            alpha,
            beta,
            baseline_volatility,
            baseline_correlation,
            gamma,
            delta
        )
        return idx, sr, params, weights, vol, mdd

    # Execute parameter evaluations in parallel
    results = Parallel(n_jobs=n_jobs)(
        delayed(evaluate_params)(idx, params) for idx, params in enumerate(ParameterGrid(param_grid))
    )

    # Sort results by Sharpe ratio and track progress
    results = sorted(results, key=lambda x: x[1], reverse=True)
    iterations = [result[0] for result in results]
    sharpe_ratios = [result[1] for result in results]

    # Extract the best result
    best_result = results[0]
    best_params = best_result[2]
    best_weights = best_result[3]
    best_sr = best_result[1]
    best_vol = best_result[4]
    best_mdd = best_result[5]

    # Plot Sharpe Ratio improvement using Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=iterations,
        y=sharpe_ratios,
        mode='lines+markers',
        line=dict(color='blue'),
        marker=dict(size=8),
        name='Sharpe Ratio'
    ))

    fig.update_layout(
        title={
            'text': "Sharpe Ratio Improvement During Higher-Order DSHRP Optimization",
            'x': 0.5,
            'xanchor': 'center'
        },
        xaxis_title="Iteration",
        yaxis_title="Sharpe Ratio",
        font=dict(size=12),
        height=600,
        width=1000,
        margin=dict(l=20, r=20, t=50, b=20),
        template="plotly_white",
        legend=dict(font=dict(size=12))
    )

    fig.write_image(file_name, scale=3)

    fig.show()

    # Display the optimal parameters and corresponding metrics
    print("\nOptimal Higher-Order DSHRP Parameters:")
    print(f"alpha: {best_params['alpha']}, beta: {best_params['beta']}, "
          f"baseline_volatility: {best_params['baseline_volatility']}, "
          f"baseline_correlation: {best_params['baseline_correlation']}, "
          f"gamma: {best_params['gamma']}, delta: {best_params['delta']}")
    print(f"Optimal Portfolio Weights: {best_weights}")
    print(f"Optimal Sharpe Ratio: {best_sr:.2%}")
    print(f"Portfolio Volatility: {best_vol:.2%}")
    print(f"Maximum Drawdown: {best_mdd:.2%}")

    return best_params, best_weights, best_sr, best_vol, best_mdd

### Define HOM DS-HRP Parameter Grid

In [13]:
param_grid_hom_dshrp = {
    'alpha': np.arange(0, 2.5, 0.5),
    'beta': np.arange(1, 6, 1),
    'baseline_volatility': np.arange(0.03, 0.36, 0.06),
    'baseline_correlation': np.arange(0.1, 0.6, 0.1),
    'gamma': [5, 10, 20],
    'delta': [5, 10, 20]
}

# Empirical Results

## Original Empirical Results

### Preprocessing and Data Preparation (In-Sample)

In [ ]:
# Test stationarity by ADF for asset log returns
def adf_test(series, asset_name):
    result = adfuller(series, autolag='AIC')
    print(f'\nADF Test for {asset_name}:')
    print(f'Test Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    print(f'#Lags Used: {result[2]}')
    print(f'#Observations: {result[3]}')
    if result[1] < 0.01:
        print(f'{asset_name} is stationary (Reject H0)')
    else:
        print(f'{asset_name} is not stationary (Fail to Reject H0)')

for asset in log_returns_20to21.columns:
    adf_test(log_returns_20to21[asset], asset)


ADF Test for FTM:
Test Statistic: -8.1237
p-value: 0.0000
#Lags Used: 4
#Observations: 360
FTM is stationary (Reject H0)

ADF Test for RUNE:
Test Statistic: -8.8273
p-value: 0.0000
#Lags Used: 3
#Observations: 361
RUNE is stationary (Reject H0)

ADF Test for CRV:
Test Statistic: -20.4601
p-value: 0.0000
#Lags Used: 0
#Observations: 364
CRV is stationary (Reject H0)

ADF Test for SUSHI:
Test Statistic: -7.7935
p-value: 0.0000
#Lags Used: 5
#Observations: 359
SUSHI is stationary (Reject H0)

ADF Test for YFI:
Test Statistic: -7.6908
p-value: 0.0000
#Lags Used: 7
#Observations: 357
YFI is stationary (Reject H0)


In [ ]:
def adf_test_with_rolling_plot(series, asset_name):
    # Perform ADF test
    result = adfuller(series, autolag='AIC')
    test_stat = result[0]
    p_value = result[1]
    title = f"{asset_name} - ADF Test: Test Stat={test_stat:.4f}, p-value={p_value:.4f}"

    # Calculate rolling statistics
    rolmean = series.rolling(window=30).mean()  # 30-day rolling mean
    rolstd = series.rolling(window=30).std()    # 30-day rolling std

    # Determine stationarity status
    adf_result_text = f"{asset_name} is {'stationary (Reject H0)' if p_value < 0.01 else 'not stationary (Fail to Reject H0)'}"
    return series, rolmean, rolstd, title, adf_result_text

# Create a 2x3 grid of subplots
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=log_returns_20to21.columns[:5],
    horizontal_spacing=0.05,  # Reduce horizontal spacing between subplots
    vertical_spacing=0.1      # Reduce vertical spacing between subplots
)

# Plot each asset in subplots
for i, asset in enumerate(log_returns_20to21.columns[:5]):
    row, col = divmod(i, 3)
    row += 1  # Adjust for Plotly's 1-indexing
    col += 1

    # Get the series and calculated values
    series = log_returns_20to21[asset].dropna()  # Ensure no NaNs
    series, rolmean, rolstd, title, adf_result_text = adf_test_with_rolling_plot(series, asset)

    # Add original series
    fig.add_trace(go.Scatter(
        x=series.index, y=series, mode='lines', name='Original Log Returns',
        line=dict(color='steelblue'), showlegend=(i == 0)
    ), row=row, col=col)

    # Add rolling mean
    fig.add_trace(go.Scatter(
        x=rolmean.index, y=rolmean, mode='lines', name='Rolling Mean',
        line=dict(color='gold'), showlegend=(i == 0)
    ), row=row, col=col)

    # Add rolling std
    fig.add_trace(go.Scatter(
        x=rolstd.index, y=rolstd, mode='lines', name='Rolling Std',
        line=dict(color='firebrick'), showlegend=(i == 0)
    ), row=row, col=col)

    # Add ADF result annotation
    fig.add_annotation(
        xref="x domain", yref="y domain",
        x=0.5, y=0.9, showarrow=False,
        text=adf_result_text, font=dict(color="seagreen" if "stationary" in adf_result_text else "firebrick"),
        row=row, col=col
    )

# Hide the last empty subplot if there are only 5 assets
if len(log_returns_20to21.columns) < 6:
    fig.update_xaxes(visible=False, row=2, col=3)
    fig.update_yaxes(visible=False, row=2, col=3)

# Update layout
fig.update_layout(
    font=dict(size=12),
    height=1000, width=1400,  # Increase height and width for larger subplots
    title={
        'text': "ADF Test and Rolling Statistics for Five Assets",
        'x': 0.5,
        'xanchor': 'center'
    },
    legend=dict(
        font=dict(size=12),
        title="Legend",
        orientation="h",
        yanchor="bottom",
        y=-0.1,  # Adjust legend position to reduce margin
        xanchor="center",
        x=0.5
    ),
    margin=dict(l=20, r=20, t=80, b=20)  # Reduce plot margins
)

fig.write_image("/content/drive/My Drive/data/20to21_startionary_plotly.png", scale=3)

# Show plot
fig.show()

To ensure the suitability of the dataset for time-series analysis, we conducted the Augmented Dickey-Fuller (ADF) test on the in-sample dataset (October 1, 2020 – September 30, 2021) to assess stationarity. Stationarity is a fundamental assumption for time-series models, implying that statistical properties such as mean, variance, and autocovariance remain constant over time.

The results of the ADF test are summarized in the code outputs, where the p-values for all five cryptocurrencies are below the 1\% significance level. This indicates that the null hypothesis of non-stationarity is rejected for all assets, confirming that their log return series are stationary.

Additionally, the figure illustrates the rolling mean and standard deviation for each cryptocurrency asset over the in-sample period. While the rolling mean remains relatively stable across the observed period, the rolling standard deviation fluctuates around a value close to 0.1, reflecting moderate volatility in the assets' returns.

These preprocessing steps and stationarity validations form the foundation for applying the DS-HRP methodology, ensuring robust and reliable results in subsequent portfolio optimization and risk analysis.

### Optimal Parameters (In-Sample, Out-of-Sample, Sensitivity)

In [ ]:
# In-sample
optimal_params_20to21, optimal_weights_20to21, optimal_sr_20to21, optimal_vol_20to21, optimal_mdd_20to21 = find_optimal_params(log_returns_20to21, rf_rate_20to21, param_grid_dshrp)
print("2020-2021 DeFi Optimal Parameters:", optimal_params_20to21)

# Out-of-sample
optimal_params_21to22, optimal_weights_21to22, optimal_sr_21to22, optimal_vol_21to22, optimal_mdd_21to22 = find_optimal_params(log_returns_21to22, rf_rate_21to22, param_grid_dshrp)
print("2021-2022 DeFi Optimal Parameters:", optimal_params_21to22)

optimal_params_22to23, optimal_weights_22to23, optimal_sr_22to23, optimal_vol_22to23, optimal_mdd_22to23 = find_optimal_params(log_returns_22to23, rf_rate_22to23, param_grid_dshrp)
print("2022-2023 DeFi Optimal Parameters:", optimal_params_22to23)

optimal_params_23to24, optimal_weights_23to24, optimal_sr_23to24, optimal_vol_23to24, optimal_mdd_23to24 = find_optimal_params(log_returns_23to24, rf_rate_23to24, param_grid_dshrp)
print("2023-2024 DeFi Optimal Parameters:", optimal_params_23to24)

# Bitcoin
optimal_params_bit23to24, optimal_weights_bit23to24, optimal_sr_bit23to24, optimal_vol_bit23to24, optimal_mdd_bit23to24 = find_optimal_params(log_returns_bit23to24, rf_rate_23to24, param_grid_dshrp)
print("2023-2024 Bitcoin Optimal Parameters:", optimal_params_bit23to24)

# 4 Year
optimal_params_20to24, optimal_weights_20to24, optimal_sr_20to24, optimal_vol_20to24, optimal_mdd_20to24 = find_optimal_params(log_returns_20to24, rf_rate_20to24, param_grid_dshrp)
print("2020-2024 DeFi Optimal Parameters:", optimal_params_20to24)


Optimal DSHRP Parameters:
t: 0.1, alpha: 0.9, kappa_0: 48
Optimal Portfolio Weights: [0.19845618 0.2405945  0.19933861 0.17624414 0.18536658]
Optimal Sharpe Ratio: 13.52%
Portfolio Volatility: 3.71%
Maximum Drawdown: 82.79%
2020-2021 DeFi Optimal Parameters: {'alpha': 0.9, 'kappa_0': 48, 't': 0.1}

Optimal DSHRP Parameters:
t: 0.45000000000000007, alpha: 0.9, kappa_0: 10
Optimal Portfolio Weights: [0.07511187 0.07511749 0.07516444 0.07523336 0.69937283]
Optimal Sharpe Ratio: -3.13%
Portfolio Volatility: 3.25%
Maximum Drawdown: 92.69%
2021-2022 DeFi Optimal Parameters: {'alpha': 0.9, 'kappa_0': 10, 't': 0.45000000000000007}

Optimal DSHRP Parameters:
t: 0.1, alpha: 1.0, kappa_0: 41
Optimal Portfolio Weights: [0.17742314 0.23852222 0.18076885 0.21510349 0.1881823 ]
Optimal Sharpe Ratio: 0.13%
Portfolio Volatility: 1.66%
Maximum Drawdown: 60.33%
2022-2023 DeFi Optimal Parameters: {'alpha': 1.0, 'kappa_0': 41, 't': 0.1}

Optimal DSHRP Parameters:
t: 0.1, alpha: 0.9, kappa_0: 47
Optimal Po

In [17]:
# optimal_params_20to21 = {'alpha': 0.9, 'kappa_0': 48, 't': 0.1}
# optimal_params_21to22 = {'alpha': 0.9, 'kappa_0': 10, 't': 0.45000000000000007}
# optimal_params_22to23 = {'alpha': 0.30000000000000004, 'kappa_0': 10, 't': 0.40000000000000013}
# optimal_params_23to24 = {'alpha': 1.1, 'kappa_0': 36, 't': 0.1}
# optimal_params_bit23to24 = {'alpha': 0.4, 'kappa_0': 26, 't': 0.5500000000000002}
# optimal_params_20to24 = {'alpha': 1.0, 'kappa_0': 42, 't': 0.1}

### Time-Varying Shrinkage Factor and Capital Allocation Across Clusters (In-Sample)

In [ ]:
# Function to calculate time-varying shrinkage factor
def calculate_shrinkage_factor(cov_matrix, alpha, kappa_0):
    eigenvalues = np.linalg.eigvals(cov_matrix)
    condition_number = np.max(eigenvalues) / np.min(eigenvalues)
    shrinkage_factor = 1 / (1 + np.exp(-alpha * (condition_number - kappa_0)))
    return shrinkage_factor

In [ ]:
# Function to plot shrinkage factors using Plotly
def plot_shrinkage_factors(returns, t, alpha, kappa_0, window=30):
    shrinkage_factors = pd.DataFrame(index=returns.index[window-1:], columns=returns.columns)
    for end in range(window, len(returns)):
        rolling_returns = returns.iloc[end-window:end]
        corr_matrix = rolling_returns.corr()
        distance_matrix = np.sqrt(0.5 * (1 - corr_matrix))
        link = linkage(squareform(distance_matrix), method='complete')
        cluster_labels = fcluster(link, t, criterion='distance')

        for label in np.unique(cluster_labels):
            assets_in_cluster = rolling_returns.columns[cluster_labels == label]
            cluster_returns = rolling_returns[assets_in_cluster]
            lw = LedoitWolf()
            cov_matrix = lw.fit(cluster_returns).covariance_
            shrinkage_factors.loc[rolling_returns.index[-1], assets_in_cluster] = calculate_shrinkage_factor(cov_matrix, alpha, kappa_0)

    fig = make_subplots(rows=1, cols=3, subplot_titles=("Shrinkage Factors Across Clusters",
                                                        "Capital Allocation Across Clusters",
                                                        "30-Day Rolling Volatility"))

    # Plot shrinkage factors with dynamic axis range
    min_val, max_val = shrinkage_factors.min().min(), shrinkage_factors.max().max()
    for col in shrinkage_factors.columns:
        fig.add_trace(
            go.Scatter(x=shrinkage_factors.index, y=shrinkage_factors[col],
                       mode='lines', name=f"{col} Shrinkage Factors"),
            row=1, col=1
        )
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text="Shrinkage Factor (λ)", range=[min_val, max_val], row=1, col=1)
    return fig

# Function to plot capital allocation using Plotly
def plot_capital_allocation(returns, t, alpha, kappa_0, window=30):
    capital_allocations = pd.DataFrame(index=returns.index[window-1:], columns=returns.columns)
    for end in range(window, len(returns)):
        rolling_returns = returns.iloc[end-window:end]
        corr_matrix = rolling_returns.corr()
        distance_matrix = np.sqrt(0.5 * (1 - corr_matrix))
        link = linkage(squareform(distance_matrix), method='complete')
        cluster_labels = fcluster(link, t, criterion='distance')

        cluster_allocations = {}
        for label in np.unique(cluster_labels):
            assets_in_cluster = rolling_returns.columns[cluster_labels == label]
            cluster_returns = rolling_returns[assets_in_cluster]
            vol = cluster_returns.std().mean()
            cluster_allocations[label] = 1 / vol if vol > 0 else 0

        total_alloc = sum(cluster_allocations.values())
        for label, alloc in cluster_allocations.items():
            capital_allocations.loc[rolling_returns.index[-1], rolling_returns.columns[cluster_labels == label]] = alloc / total_alloc

    # Plot capital allocation with dynamic axis range
    min_val, max_val = capital_allocations.min().min(), capital_allocations.max().max()
    for col in capital_allocations.columns:
        fig.add_trace(
            go.Scatter(x=capital_allocations.index, y=capital_allocations[col],
                       mode='lines', name=f"{col} Capital Allocation"),
            row=1, col=2
        )
    fig.update_xaxes(title_text="Date", row=1, col=2)
    fig.update_yaxes(title_text="Capital Allocation", range=[min_val, max_val], row=1, col=2)
    return fig

# Function to plot rolling volatility using Plotly
def plot_rolling_volatility(returns, window=30):
    rolling_volatility = returns.rolling(window=window).std() * (252 ** 0.5)
    min_val, max_val = rolling_volatility.min().min(), rolling_volatility.max().max()

    for asset in returns.columns:
        fig.add_trace(
            go.Scatter(x=rolling_volatility.index, y=rolling_volatility[asset],
                       mode='lines', name=f'{asset} Volatility'),
            row=1, col=3
        )
    fig.update_xaxes(title_text="Date", row=1, col=3)
    fig.update_yaxes(title_text="Annualized Volatility", range=[min_val, max_val], row=1, col=3)
    return fig

#### Pre-defined Parameters

In [ ]:
# Combined Plot with Plotly Subplots
fig = plot_shrinkage_factors(log_returns_20to21, t=0.3, alpha=1, kappa_0=35)
fig = plot_capital_allocation(log_returns_20to21, t=0.3, alpha=1, kappa_0=35)
fig = plot_rolling_volatility(log_returns_20to21, window=30)

# Update layout to make each subplot 1:1
fig.update_layout(
    font=dict(size=12),
    title={"text": "Time-Varying Shrinkage Factors, Capital Allocation, and Volatility", "x": 0.5, "xanchor": "center"},
    height=500,  # Adjust height to ensure squares
    width=1500,  # Width should be adjusted for 3 square plots side-by-side
    legend=dict(font=dict(size=12), title="Legend", orientation="h", yanchor="bottom", y=-0.5, xanchor="center", x=0.5),
    margin=dict(l=20, r=20, t=80, b=20)
)

# Set each subplot with 1:1 aspect ratio
for i in range(1, 4):
    fig.update_yaxes(scaleanchor="x", scaleratio=1, row=1, col=i)

fig.write_image("/content/drive/My Drive/data/20to21_sf_ca_vol_cons_plotly.png", scale=3)

fig.show()


The dynamic shrinkage mechanism of DS-HRP was implemented on the correlation matrix derived from the asset returns. We clustered the in-sample five DeFi assets into groups based on their pairwise correlations and applied the dynamic shrinkage factor $\lambda_c$ to each cluster-specific covariance matrix. The shrinkage factor was adjusted dynamically based on the condition number $\kappa_c$ of each cluster's covariance matrix, with higher shrinkage applied to more unstable clusters exhibiting larger condition numbers.

Using the constant parameters $t=0.3$, $\alpha=1$, and $\kappa_0=35$, the first subplot illustrates the time-varying behavior of the shrinkage factor across clusters. Clusters experiencing elevated condition numbers, often associated with market stress or increased correlation instability, received higher shrinkage adjustments, particularly during mid-2021's DeFi market boom.

After applying dynamic shrinkage, quasi-diagonalization was performed on the adjusted covariance matrices to minimize cross-cluster correlations by grouping highly correlated assets along the diagonal. Following this transformation, recursive bisection was used to allocate capital across clusters based on risk-parity principles. This allocation approach ensures that clusters with lower volatility levels receive higher capital allocations, optimizing the risk distribution.

The second subplot illustrates capital allocation across clusters. The allocation dynamically adjusted in response to both shrinkage factors and volatility levels, aligning with the risk-parity framework. Notably, during July 2021, Yearn.Finance (YFI), a lower-volatility asset, received higher capital allocations, while higher-volatility assets such as ThorChain (RUNE) and Fantom (FTM) experienced reduced allocations.

The third subplot displays the annualized 30-day rolling volatility of each asset, emphasizing periods of increased risk throughout the studied period. Peaks in volatility, particularly during mid-2021, align with observable shifts in both shrinkage activity and capital allocation, highlighting the model's responsiveness to changing risk environments.

Together, these three subplots—shrinkage factors, capital allocation, and rolling volatility—reveal the dynamic interplay between shrinkage adjustments, capital allocation strategies, and market volatility patterns. The alignment of shrinkage activity with volatility peaks and capital reallocation reflects the DS-HRP model's adaptability in managing risk and maintaining portfolio stability across varying market conditions.

#### Optimal Parameters

In [ ]:
# Combined Plot with Plotly Subplots
fig = plot_shrinkage_factors(log_returns_20to21, t=optimal_params_20to21['t'], alpha=optimal_params_20to21['alpha'], kappa_0=optimal_params_20to21['kappa_0'])
fig = plot_capital_allocation(log_returns_20to21, t=optimal_params_20to21['t'], alpha=optimal_params_20to21['alpha'], kappa_0=optimal_params_20to21['kappa_0'])
fig = plot_rolling_volatility(log_returns_20to21, window=30)

# Update layout to make each subplot 1:1
fig.update_layout(
    font=dict(size=12),
    title={"text": "Time-Varying Shrinkage Factors, Capital Allocation, and Volatility", "x": 0.5, "xanchor": "center"},
    height=500,  # Adjust height to ensure squares
    width=1500,  # Width should be adjusted for 3 square plots side-by-side
    legend=dict(font=dict(size=12), title="Legend", orientation="h", yanchor="bottom", y=-0.5, xanchor="center", x=0.5),
    margin=dict(l=20, r=20, t=80, b=20)
)

# Set each subplot with 1:1 aspect ratio
for i in range(1, 4):
    fig.update_yaxes(scaleanchor="x", scaleratio=1, row=1, col=i)

fig.write_image("/content/drive/My Drive/data/20to21_sf_ca_vol_opt_plotly.png", scale=3)

fig.show()

To enable a more accurate comparison of DS-HRP against traditional HRP and Markowitz's mean-variance methods, we conducted an optimal configuration for DS-HRP to maximize performance. A grid search algorithm systematically explored combinations of parameters: $t$ (distance threshold for clustering), $\alpha$ (sensitivity factor for dynamic shrinkage), and $\kappa_0$ (threshold for shrinkage adjustment). For each combination, portfolio weights were calculated, and key performance metrics—including Sharpe ratio, portfolio volatility, and maximum drawdown—were evaluated. The parameter set maximizing the Sharpe ratio was selected to ensure optimal risk-adjusted returns.

The search yielded the optimal parameters: $t=0.1$, $\alpha=0.9$, $\kappa_0=48$ for the in-sample dataset. The plot illustrates the shrinkage factors, capital allocation, and 30-day rolling annualized volatility under these optimized settings.

The first subplot displays the time-varying shrinkage factors across clusters. Compared to the predefined parameters, the optimized shrinkage factors exhibit smoother adjustments, avoiding sharp spikes during periods of heightened market stress, such as mid-2021. This refinement suggests improved stability in covariance estimation, particularly in turbulent market conditions.

The second subplot presents capital allocation across clusters. The optimized configuration results in smoother transitions and fewer abrupt shifts in allocation. Clusters with lower volatility, such as Yearn.Finance (YFI), consistently receive higher capital allocations, while more volatile clusters like ThorChain (RUNE) and Fantom (FTM) maintain reduced allocations. This dynamic adjustment aligns capital allocation with evolving risk profiles, improving overall portfolio stability.

The third subplot illustrates 30-day rolling annualized volatility across assets. While the general volatility patterns remain consistent with the predefined parameters, the optimized settings exhibit better control over volatility spikes, especially during periods of elevated market uncertainty. This improved volatility management contributes to more predictable risk profiles.

In summary, parameter optimization resulted in three key improvements:
1. Shrinkage Factors: More stable adjustments, reducing extreme variations during stress periods.
2. Capital Allocation: Smoother and more consistent capital distribution across clusters.
3. Volatility Management: Improved control over sharp volatility spikes, maintaining portfolio resilience.

Compared to the predefined configuration, the optimized model demonstrates enhanced adaptability and robustness. These results highlight the optimized DS-HRP model's ability to dynamically adapt to market conditions, offering improved resilience and risk control in highly volatile environments.

### Performance Evaluation (In-Sample, Out-of-Sample, Sensitivity)

In [ ]:
def analyze_performance(returns, rf_rate, optimal_params, file_name):
    """Analyze and plot the performance of DS-HRP, HRP, and Markowitz's mean-variance for a specific rolling year.

    Parameters:
    - returns: DataFrame of log returns for the period.
    - rf_rate: Series of risk-free rates for the period.
    - optimal_params: Dictionary containing the optimal parameters (t, alpha, kappa_0) for DS-HRP.
    - file_name: The name of the file to save the plot as.

    Returns:
    - metrics_df: DataFrame with the calculated performance metrics for each method.
    - An interactive Plotly plot comparing the Sharpe Ratio, Volatility, and Max Drawdown of the methods.
    """

    # DS-HRP
    t = optimal_params['t']
    alpha = optimal_params['alpha']
    kappa_0 = optimal_params['kappa_0']
    dshrp_weights_val, dshrp_sr, dshrp_vol, dshrp_mdd = dshrp_weights(returns, rf_rate, t, alpha, kappa_0)
    print(f"DSHRP Weights: {dshrp_weights_val}")

    # HRP
    hrp_weights_val = hrp_weights(returns)
    hrp_sr, hrp_vol, hrp_mdd = performance(hrp_weights_val, returns, rf_rate)
    print(f"HRP Weights: {hrp_weights_val}")

    # Markowitz's
    mv_weights_val = mv_weights(returns)
    mv_sr, mv_vol, mv_mdd = performance(mv_weights_val, returns, rf_rate)
    print(f"Markowitz Weights: {mv_weights_val}")

    # Print Results
    print(f"DSHRP: Sharpe Ratio = {dshrp_sr:.4%}, Volatility = {dshrp_vol:.4%}, MDD = {dshrp_mdd:.2%}")
    print(f"Traditional HRP: Sharpe Ratio = {hrp_sr:.4%}, Volatility = {hrp_vol:.4%}, MDD = {hrp_mdd:.2%}")
    print(f"Markowitz: Sharpe Ratio = {mv_sr:.4%}, Volatility = {mv_vol:.4%}, MDD = {mv_mdd:.2%}")

    # Create DataFrame for Visualization
    metrics_df = pd.DataFrame({
        "Method": ["DSHRP", "Traditional HRP", "Markowitz"],
        "Sharpe Ratio": [dshrp_sr, hrp_sr, mv_sr],
        "Volatility": [dshrp_vol, hrp_vol, mv_vol],
        "Max Drawdown": [dshrp_mdd, hrp_mdd, mv_mdd]
    })

    # Create subplots with three separate plots for Sharpe Ratio, Volatility, and Max Drawdown
    fig = make_subplots(rows=1, cols=3, subplot_titles=["Sharpe Ratio", "Volatility", "Max Drawdown"])

    # Add Sharpe Ratio bar chart
    fig.add_trace(go.Bar(
        x=metrics_df["Method"],
        y=metrics_df["Sharpe Ratio"],
        name="Sharpe Ratio",
        marker_color="gold"
    ), row=1, col=1)

    # Add Volatility bar chart
    fig.add_trace(go.Bar(
        x=metrics_df["Method"],
        y=metrics_df["Volatility"],
        name="Volatility",
        marker_color="seagreen"
    ), row=1, col=2)

    # Add Max Drawdown bar chart
    fig.add_trace(go.Bar(
        x=metrics_df["Method"],
        y=metrics_df["Max Drawdown"],
        name="Max Drawdown",
        marker_color="firebrick"
    ), row=1, col=3)

    # Update layout and set individual y-axis ranges for each subplot
    fig.update_layout(
        font=dict(size=12),
        height=500,
        width=1200,
        margin=dict(l=20, r=20, t=30, b=30),
        showlegend=False
    )

    fig.show()

    # Save the plot using the provided file name
    fig.write_image(file_name, scale=3)

    return metrics_df

#### In-Sample: October 1, 2020 to September 30, 2021

In [ ]:
metrics_20to21 = analyze_performance(log_returns_20to21, rf_rate_20to21, optimal_params_20to21, file_name="/content/drive/My Drive/data/20to21_sr_vol_mdd_plotly.png")

DSHRP Weights: [0.19845618 0.2405945  0.19933861 0.17624414 0.18536658]
HRP Weights: [0.17670507 0.17670507 0.21552995 0.21552995 0.21552995]
Markowitz Weights: [0.09956865 0.17715193 0.0858558  0.07416097 0.56326265]
DSHRP: Sharpe Ratio = 13.5158%, Volatility = 3.7094%, MDD = 82.79%
Traditional HRP: Sharpe Ratio = 12.9712%, Volatility = 3.6422%, MDD = 81.75%
Markowitz: Sharpe Ratio = 12.5219%, Volatility = 3.8660%, MDD = 81.29%


**Sharpe Ratio:** DS-HRP achieved the highest Sharpe Ratio (13.52\%), outperforming both HRP (12.97\%) and Markowitz (12.52\%). This superior risk-adjusted return highlights the effectiveness of DS-HRP's dynamic shrinkage adjustment mechanism in stabilizing the covariance matrix and managing risk across diverse market conditions. The optimized parameters ($t=0.1$, $\alpha=0.9$, $\kappa_0=48$) played a pivotal role in ensuring that capital allocation aligned with risk dynamics, particularly during periods of heightened volatility.

**Volatility:** The portfolio volatility under DS-HRP (3.71\%) remained well-balanced, slightly higher than HRP (3.64\%) but notably lower than Markowitz (3.87\%). DS-HRP's ability to dynamically adjust shrinkage factors across asset clusters helped control volatility without over-concentrating allocations in any single asset class.

**Maximum Drawdown:** DS-HRP recorded a drawdown of 82.79\%, closely aligned with HRP (81.75\%) and slightly outperforming Markowitz (81.29\%). Although drawdowns were substantial across all methods, DS-HRP demonstrated an ability to cushion extreme losses better than Markowitz’s approach while maintaining a more adaptive allocation strategy.

**Overall:** Compared to HRP and Markowitz, DS-HRP delivered more stable risk-adjusted returns, efficiently managing both systemic volatility and asset-specific risks. This adaptability is especially valuable in the cryptocurrency market, where volatility is frequent and severe.

#### Out-of-Sample: October 1, 2021 to September 30, 2022

In [ ]:
metrics_21to22 = analyze_performance(log_returns_21to22, rf_rate_21to22, optimal_params_21to22, file_name="/content/drive/My Drive/data/21to22_sr_vol_mdd_plotly.png")

DSHRP Weights: [0.07511187 0.07511749 0.07516444 0.07523336 0.69937283]
HRP Weights: [0.15268755 0.15268755 0.23154163 0.23154163 0.23154163]
Markowitz Weights: [0.06215563 0.0089114  0.09448252 0.3153363  0.51911415]
DSHRP: Sharpe Ratio = -3.1347%, Volatility = 3.2528%, MDD = 92.69%
Traditional HRP: Sharpe Ratio = -6.5114%, Volatility = 2.6625%, MDD = 93.78%
Markowitz: Sharpe Ratio = -3.9461%, Volatility = 3.0635%, MDD = 93.29%


**Sharpe Ratio:** All three methods reported negative values, reflecting challenging market conditions. DS-HRP recorded a Sharpe Ratio of -3.13\%, outperforming Markowitz (-3.95\%) and significantly surpassing HRP (-6.51\%). Despite unfavorable market conditions characterized by declining asset values and heightened volatility, DS-HRP demonstrated a relatively better ability to preserve risk-adjusted returns through its dynamic shrinkage mechanism.

**Volatility:** DS-HRP exhibited a slightly higher volatility of 3.25\% compared to HRP (2.66\%) but remained relatively close to Markowitz (3.06\%). The elevated volatility across all methods suggests market-wide turbulence during this period. However, DS-HRP's volatility level reflects its dynamic risk adjustment, which avoided over-concentration in low-volatility assets while still managing cluster-specific risks effectively.

**Maximum Drawdown:** DS-HRP recorded a drawdown of 92.69\%, slightly outperforming Markowitz (93.29\%) and HRP (93.78\%). While drawdowns remained severe across all strategies, DS-HRP’s marginally lower drawdown indicates a slightly better capacity to mitigate extreme losses during prolonged downturns. This resilience can be attributed to the dynamic shrinkage adjustments, which redistributed capital more adaptively across clusters in response to increased risk.

**Overall:** The DS-HRP approach showed relative robustness amid challenging market conditions during this out-of-sample period. While all three methodologies faced significant performance deterioration, DS-HRP’s dynamic shrinkage adjustments and adaptive capital allocation framework provided a modest edge in risk-adjusted returns and drawdown management.

#### Out-of-Sample: October 1, 2022 to September 30, 2023

In [ ]:
metrics_22to23 = analyze_performance(log_returns_22to23, rf_rate_22to23, optimal_params_22to23, file_name="/content/drive/My Drive/data/22to23_sr_vol_mdd_plotly.png")

DSHRP Weights: [0.17742314 0.23852222 0.18076885 0.21510349 0.1881823 ]
HRP Weights: [0.1856859  0.1856859  0.20954273 0.20954273 0.20954273]
Markowitz Weights: [0.         0.30823997 0.00529367 0.02255691 0.66390945]
DSHRP: Sharpe Ratio = 0.1313%, Volatility = 1.6575%, MDD = 60.33%
Traditional HRP: Sharpe Ratio = -0.1050%, Volatility = 1.6578%, MDD = 61.48%
Markowitz: Sharpe Ratio = 0.1107%, Volatility = 2.1200%, MDD = 53.63%


**Sharpe Ratio:** DS-HRP achieved a Sharpe Ratio of 0.13\%, outperforming both HRP (-0.11\%) and Markowitz (0.11\%). This indicates that DS-HRP was able to deliver slightly better risk-adjusted returns compared to Markowitz and performed notably better than HRP, which recorded a negative Sharpe Ratio. Despite relatively subdued returns during this period, DS-HRP demonstrated resilience, attributed to its dynamic covariance shrinkage mechanism, which helped stabilize risk contributions from individual clusters.

**Volatility:** Portfolio volatility was nearly identical for DS-HRP (1.66\%) and HRP (1.66\%), while Markowitz exhibited a higher volatility of 2.12\%. The lower volatility in DS-HRP and HRP reflects their ability to control risk exposure effectively, with DS-HRP benefiting from optimized parameters ($t=0.1$, $\alpha=1.0$, $\kappa_0=41$) to better respond to cluster-specific instabilities. Markowitz's higher volatility suggests an increased sensitivity to asset-specific risks and suboptimal diversification during this period.

**Maximum Drawdown:** Maximum Drawdown was the lowest for Markowitz (53.63\%), followed by DS-HRP (60.33\%), and HRP (61.48\%). While Markowitz demonstrated better drawdown performance, this result comes at the cost of increased portfolio volatility. DS-HRP, while experiencing a moderately higher drawdown than Markowitz, maintained more consistent volatility and risk-adjusted returns, demonstrating improved robustness in managing tail risks during periods of market stress.

**Overall:** DS-HRP exhibited a balanced performance across all three metrics—Sharpe Ratio, Volatility, and Maximum Drawdown—outperforming HRP in both Sharpe Ratio and Maximum Drawdown while maintaining lower volatility compared to Markowitz.

#### Out-of-Sample: October 1, 2023 to September 30, 2024

In [ ]:
metrics_23to24 = analyze_performance(log_returns_23to24, rf_rate_23to24, optimal_params_23to24, file_name="/content/drive/My Drive/data/23to24_sr_vol_mdd_plotly.png")

DSHRP Weights: [0.18212457 0.17498203 0.20625413 0.18514661 0.25149267]
HRP Weights: [0.1498224  0.1498224  0.23345173 0.23345173 0.23345173]
Markowitz Weights: [0.0785409  0.00227255 0.2578566  0.03484128 0.62648868]
DSHRP: Sharpe Ratio = 2.7859%, Volatility = 1.9161%, MDD = 73.22%
Traditional HRP: Sharpe Ratio = 2.1690%, Volatility = 1.9259%, MDD = 73.97%
Markowitz: Sharpe Ratio = 1.4115%, Volatility = 2.0768%, MDD = 69.53%


**Sharpe Ratio:** DS-HRP achieved the highest Sharpe Ratio at 2.79\%, outperforming both HRP (2.17\%) and Markowitz (1.41\%). This result highlights DS-HRP's superior ability to balance risk and return effectively. The dynamic shrinkage mechanism allowed for more adaptive covariance estimation, optimizing the allocation across asset clusters in response to changing market dynamics. Markowitz, despite its theoretical optimality under mean-variance assumptions, underperformed due to its sensitivity to estimation errors in both return and covariance inputs.

**Volatility:** DS-HRP recorded a portfolio volatility of 1.92\%, slightly lower than HRP (1.93\%) and notably lower than Markowitz (2.08\%). This indicates that DS-HRP maintained portfolio stability while dynamically adjusting capital allocation to mitigate exposure to high-risk assets. The stability observed in DS-HRP's volatility profile reflects its capacity to smooth out extreme fluctuations during turbulent periods, a key advantage over Markowitz’s mean-variance approach.

**Maximum Drawdown:** DS-HRP also demonstrated moderate performance in terms of maximum drawdown, with a recorded value of 73.22\%, slightly better than HRP (73.97\%) but falling behind Markowitz (69.53\%). While Markowitz showed a smaller drawdown, it did so at the expense of higher volatility and lower Sharpe Ratio. DS-HRP, on the other hand, balanced drawdown and volatility more effectively, showcasing its resilience in managing downside risk without excessive exposure to concentrated risk clusters.

**Overall:** DS-HRP outperformed HRP and Markowitz in delivering a higher Sharpe Ratio with competitive volatility and manageable drawdown levels. Markowitz’s slightly better maximum drawdown came at the cost of higher volatility and weaker risk-adjusted returns. Traditional HRP, while offering decent performance, lagged in Sharpe Ratio and exhibited marginally higher drawdowns than DS-HRP.

### Macroeconomic Factors and Volatility of DeFi Market

In [ ]:
# Ensure "Date" columns are in datetime format
DEFITVL_df["Date"] = pd.to_datetime(DEFITVL_df["Date"])
BCOM_df["Date"] = pd.to_datetime(BCOM_df["Date"])

# Set the Date column as the index and select relevant columns
DEFITVL = DEFITVL_df.set_index("Date")[["Total"]].rename(columns={"Total": "Price"})
BCOM = BCOM_df.set_index("Date")[["PX_LAST"]].rename(columns={"PX_LAST": "Price"})

# Filter date range from October 1, 2020, to September 30, 2024
DEFITVL = DEFITVL.loc["2020-10-01":"2024-09-30"]

# Reindex BCOM to match DEFITVL's index, using forward and backward fill
BCOM_reindexed = BCOM.reindex(DEFITVL.index).ffill().bfill()

# Create a Plotly figure
fig = go.Figure()

# Add DEFITVL plot
fig.add_trace(go.Scatter(
    x=DEFITVL.index,
    y=DEFITVL["Price"] / 1e9,
    mode="lines",
    name="DEFITVL ($1 × 10^9)",
    line=dict(color="lightskyblue")
))

# Add BCOM plot
fig.add_trace(go.Scatter(
    x=BCOM_reindexed.index,
    y=BCOM_reindexed["Price"],
    mode="lines",
    name="BCOM ($)",
    line=dict(color="darkseagreen")
))

# Update layout with axis ranges
fig.update_layout(
    font=dict(size=12),
    title={"text": "DeFi’s Total Value Locked (TVL) and Bloomberg Commodities Index (BCOM) <br> October 1, 2020 to September 30, 2024", "x": 0.5, "xanchor": "center"},
    xaxis=dict(
        title="Time",
        range=[DEFITVL.index.min(), DEFITVL.index.max()]  # Set x-axis range to the full date range
    ),
    yaxis=dict(
        title="Value",
        range=[0, max(DEFITVL["Price"].max() / 1e9, BCOM_reindexed["Price"].max()) * 1.1]  # Set y-axis to 10% above max value
    ),
    legend=dict(title="Legend"),
    height=500,
    width=1200,
    margin=dict(l=20, r=20, t=60, b=20)
)

fig.write_image("/content/drive/My Drive/data/20to24_defitvl_bcom_plotly.png", scale=3)

# Display the Plotly plot
fig.show()

**Boom and Peak (2021):** The surge in TVL during 2021 coincided with strong in-sample performance across all methodologies. DS-HRP, optimized with parameters ($t=0.1$, $\alpha=0.9$, $\kappa_0=48$), delivered the highest Sharpe Ratio and maintained a balanced volatility profile. This period of abundant liquidity and speculative capital inflows provided a favorable environment for dynamic strategies like DS-HRP to excel.

**Correction and Decline (2022):** The sharp decline in TVL from early 2022 mirrors the negative Sharpe ratios and elevated volatility observed in all portfolios. DS-HRP’s dynamic covariance shrinkage helped cushion some of the impacts, resulting in relatively better drawdown performance compared to HRP and Markowitz. However, the sudden tightening of liquidity and rising discount rates posed challenges across all strategies.

**Stabilization and Adaptation (2023):** During 2023, TVL stabilized at lower levels, aligning with moderate recovery in DS-HRP’s performance. DS-HRP outperformed HRP and Markowitz in Sharpe Ratio and demonstrated effective volatility control, underscoring its resilience during transitional market conditions.

**Gradual Recovery (2024):** By 2024, TVL showed an upward trend, reflecting improved market sentiment and liquidity conditions. DS-HRP delivered the highest Sharpe Ratio (2.79\%), outperforming HRP and Markowitz. Its dynamic adjustments allowed it to capture upside potential while maintaining a stable volatility profile.

### Robustness and Sensitivity Analysis

#### Bitcoin

In [ ]:
metrics_bit23to24 = analyze_performance(log_returns_bit23to24, rf_rate_23to24, optimal_params_bit23to24, file_name="/content/drive/My Drive/data/23to24bit_sr_vol_mdd_plotly.png")

DSHRP Weights: [0.04889078 0.04888474 0.04889786 0.04889272 0.0488889  0.04888638
 0.04888871 0.60886969 0.04890022]
HRP Weights: [0.10451227 0.10451227 0.10451227 0.10451227 0.11639018 0.11639018
 0.11639018 0.11639018 0.11639018]
Markowitz Weights: [1.10082403e-17 1.02690750e-16 2.58937585e-01 0.00000000e+00
 0.00000000e+00 7.41062415e-01 5.69511300e-18 0.00000000e+00
 1.76396314e-17]
DSHRP: Sharpe Ratio = 8.5546%, Volatility = 2.5054%, MDD = 74.33%
Traditional HRP: Sharpe Ratio = -0.7111%, Volatility = 1.3191%, MDD = 61.53%
Markowitz: Sharpe Ratio = 7.6554%, Volatility = 1.6165%, MDD = 34.87%


**Sharpe Ratio:** DS-HRP achieved the highest Sharpe Ratio (8.55\%), outperforming Markowitz (7.66\%) and significantly surpassing HRP (-0.71\%). This indicates DS-HRP's ability to deliver superior risk-adjusted returns even in highly volatile environments.

**Volatility:** DS-HRP displayed moderate volatility (2.51\%) compared to Markowitz (1.62\%) and HRP (1.32\%). While the Markowitz model achieved lower volatility, it often does so by over-concentrating in a few low-risk assets, reducing diversification.

**Maximum Drawdown:** DS-HRP experienced a maximum drawdown of 74.33\%, higher than Markowitz (34.87\%) but marginally worse than HRP (61.53\%). This highlights a trade-off in DS-HRP's optimization, where higher returns come at the cost of elevated drawdowns.

**Summary:** DS-HRP demonstrates strong robustness when transitioning from DeFi assets to Bitcoin-related assets.

#### Four Year Time Horizon

In [ ]:
metrics_20to24 = analyze_performance(log_returns_20to24, rf_rate_20to24, optimal_params_20to24, file_name="/content/drive/My Drive/data/20to24_sr_vol_mdd_plotly.png")

DSHRP Weights: [0.19190203 0.23994742 0.19954473 0.17856709 0.19003874]
HRP Weights: [0.16706086 0.16706086 0.22195942 0.22195942 0.22195942]
Markowitz Weights: [0.07403958 0.12169284 0.09791014 0.12069669 0.58566076]
DSHRP: Sharpe Ratio = 2.2366%, Volatility = 2.5234%, MDD = 98.59%
Traditional HRP: Sharpe Ratio = 2.0482%, Volatility = 2.4733%, MDD = 98.72%
Markowitz: Sharpe Ratio = 2.8857%, Volatility = 2.6391%, MDD = 98.65%


**Sharpe Ratio:** Over the four-year horizon, Markowitz achieved the highest Sharpe Ratio at 2.89\%, followed by DS-HRP at 2.24\%, and HRP at 2.05\%. While DS-HRP demonstrated consistent adaptability in shorter horizons, its performance slightly lagged behind Markowitz in the long run. This result suggests that DS-HRP’s dynamic adjustment mechanisms may be more effective in capturing short-term market dynamics rather than maintaining long-term optimization. In contrast, Markowitz benefited from its mean-variance framework, which emphasizes a more stable asset allocation over time.

**Volatility:** DS-HRP exhibited portfolio volatility of 2.52\%, marginally higher than HRP (2.47\%) but lower than Markowitz (2.64\%). Despite its adaptive capital allocation and dynamic covariance matrix shrinkage, DS-HRP's volatility remained well-contained across the extended time frame. Markowitz’s slightly higher volatility suggests sensitivity to estimation errors in both expected returns and covariance matrices, which can become more pronounced over longer periods.

**Maximum Drawdown:** All three methodologies experienced significant maximum drawdowns exceeding 98\%, indicating the high-risk nature of the cryptocurrency market over extended periods. DS-HRP recorded a maximum drawdown of 98.59\%, slightly better than HRP (98.72\%) but marginally worse than Markowitz (98.65\%). This result emphasizes that while DS-HRP’s dynamic adjustment reduces short-term exposure to extreme risks, it may not fully mitigate prolonged periods of significant market stress.

**Summary:** DS-HRP performs better over shorter time horizons than extended, multi-year periods.

## 4.8 Further Enhancement Empirical Results

### 4.8.3 Optimal Parameters (In-Sample)

In [ ]:
optimal_params_20to21ho, optimal_weights_20to21ho, optimal_sr_20to21ho, optimal_vol_20to21ho, optimal_mdd_20to21ho = find_optimal_params_higher_order(
    log_returns_20to21,
    rf_rate_20to21,
    param_grid_hom_dshrp,
    "/content/drive/My Drive/data/sharpe_ratio_optimization_plotly.png"
)
print("2020-2021 DeFi Optimal Parameters (HOM DS-HRP):", optimal_params_20to21ho)


Optimal Higher-Order DSHRP Parameters:
alpha: 2.0, beta: 1, baseline_volatility: 0.03, baseline_correlation: 0.5, gamma: 20, delta: 5
Optimal Portfolio Weights: [0.1732139  0.20512283 0.19373353 0.20108858 0.22684116]
Optimal Sharpe Ratio: 13.19%
Portfolio Volatility: 3.62%
Maximum Drawdown: 82.08%
2020-2021 DeFi Optimal Parameters (HOM DS-HRP): {'alpha': 2.0, 'baseline_correlation': 0.5, 'baseline_volatility': 0.03, 'beta': 1, 'delta': 5, 'gamma': 20}


In [14]:
# optimal_params_20to21ho = {'alpha': 2.0, 'beta': 1, 'baseline_volatility': 0.03, 'baseline_correlation': 0.5, 'gamma': 20, 'delta': 5}

1. In the early iterations (0–1000), the Sharpe Ratio exhibits minimal improvements, reflecting the exploratory nature of the initial parameter combinations.

2. Between 2000 and 5000 iterations, the Sharpe Ratio steadily increases as the optimization process refines parameter combinations, uncovering more effective balances of $\alpha$, $\beta$, $\sigma_0$, $\rho_0$, $\gamma$, and $\delta$.

3. After 5000 iterations, the Sharpe Ratio begins to stabilize, with occasional sharp increases signaling the discovery of parameter combinations that significantly enhance performance.

4. In the final phase ($\sim6500$ iterations), the Sharpe Ratio plateaus at its peak, indicating that the optimal configuration of higher-order moment adjustments has been achieved.

The upward trajectory in the Sharpe Ratio validates the effectiveness of the loss function in guiding the grid search toward optimal parameter configurations. By accounting for skewness ($\alpha$), kurtosis ($\beta$), volatility sensitivity ($\sigma_0$, $\gamma$), and correlation dynamics ($\rho_0$, $\delta$), the optimized HOM DS-HRP framework achieves a superior risk-return trade-off.

In summary, the optimization process successfully fine-tunes the HOM DS-HRP parameters, enabling the framework to dynamically adapt to market conditions, address non-normal return distributions, and deliver enhanced portfolio stability with improved risk-adjusted returns.

### 4.8.1 Time-Varying Shrinkage Factor and Capital Allocation Across Clusters (In-Sample)

In [ ]:
def plot_shrinkage_and_allocation(data, alpha, baseline_volatility, baseline_correlation, beta, gamma, delta, window=30):
    """
    Plot time-varying shrinkage factors across clusters, capital allocation across clusters,
    and 30-day rolling volatility of assets using Plotly.
    """
    shrinkage_factors = pd.DataFrame(index=data.index[window-1:], columns=data.columns)
    capital_allocations = pd.DataFrame(index=data.index[window-1:], columns=data.columns)

    for end in range(window, len(data)):
        rolling_data = data.iloc[end-window:end]

        # Step 1: Clustering and Distance Matrix
        corr_matrix = rolling_data.corr()
        distance_matrix = np.sqrt(0.5 * (1 - corr_matrix))
        link = linkage(squareform(distance_matrix), method='complete')
        cluster_labels = fcluster(link, t=0.7, criterion='distance')

        # Step 2: Shrinkage factors and capital allocation
        cluster_allocations = {}
        for label in np.unique(cluster_labels):
            assets_in_cluster = rolling_data.columns[cluster_labels == label]
            cluster_returns = rolling_data[assets_in_cluster]

            # Ledoit-Wolf shrinkage
            lw = LedoitWolf()
            cov_matrix = lw.fit(cluster_returns).covariance_
            shrinkage_factor = calculate_shrinkage_factor(cov_matrix, alpha, baseline_volatility)
            shrinkage_factors.loc[rolling_data.index[-1], assets_in_cluster] = shrinkage_factor

            # Calculate capital allocation as inverse of volatility
            vol = cluster_returns.std().mean()
            cluster_allocations[label] = 1 / vol if vol > 0 else 0

        # Normalize allocations
        total_alloc = sum(cluster_allocations.values())
        for label, alloc in cluster_allocations.items():
            capital_allocations.loc[rolling_data.index[-1], rolling_data.columns[cluster_labels == label]] = alloc / total_alloc

    # 30-Day Rolling Volatility
    rolling_volatility = data.rolling(window=30).std() * np.sqrt(252)

    # Create Plotly Subplots
    fig = make_subplots(rows=1, cols=3, subplot_titles=[
        "Shrinkage Factors Across Clusters",
        "Capital Allocation Across Clusters",
        "30-Day Rolling Volatility"
    ])

    # Shrinkage Factors Plot
    for col in shrinkage_factors.columns:
        fig.add_trace(go.Scatter(
            x=shrinkage_factors.index, y=shrinkage_factors[col],
            mode='lines', name=f"{col} Shrinkage Factor",
            legendgroup="shrinkage"
        ), row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text="Shrinkage Factor (λ)", row=1, col=1)

    # Capital Allocation Plot
    for col in capital_allocations.columns:
        fig.add_trace(go.Scatter(
            x=capital_allocations.index, y=capital_allocations[col],
            mode='lines', name=f"{col} Capital Allocation",
            legendgroup="allocation"
        ), row=1, col=2)
    fig.update_xaxes(title_text="Date", row=1, col=2)
    fig.update_yaxes(title_text="Capital Allocation", row=1, col=2)

    # Rolling Volatility Plot
    for col in rolling_volatility.columns:
        fig.add_trace(go.Scatter(
            x=rolling_volatility.index, y=rolling_volatility[col],
            mode='lines', name=f"{col} Rolling Volatility",
            legendgroup="volatility"
        ), row=1, col=3)
    fig.update_xaxes(title_text="Date", row=1, col=3)
    fig.update_yaxes(title_text="Annualized Volatility", row=1, col=3)

    fig.update_layout(
        font=dict(size=12),
        title={"text": "Time-Varying Shrinkage Factors, Capital Allocation, and Volatility", "x": 0.5, "xanchor": "center"},
        height=500,  # Adjust height to ensure squares
        width=1500,  # Width should be adjusted for 3 square plots side-by-side
        legend=dict(font=dict(size=12), title="Legend", orientation="h", yanchor="bottom", y=-0.6, xanchor="center", x=0.5),
        margin=dict(l=20, r=20, t=80, b=20)
    )

    fig.write_image("/content/drive/My Drive/data/20to21_sf_ca_vol_ho_plotly.png", scale=3)

    # Show Plot
    fig.show()

# Using Optimal Parameters Run in Previous Section
plot_shrinkage_and_allocation(
    log_returns_20to21,
    alpha=optimal_params_20to21ho['alpha'],
    baseline_volatility=optimal_params_20to21ho['baseline_volatility'],
    baseline_correlation=optimal_params_20to21ho['baseline_correlation'],
    beta=optimal_params_20to21ho['beta'],
    gamma=optimal_params_20to21ho['gamma'],
    delta=optimal_params_20to21ho['delta'],
    window=30
)

In [ ]:
# # Using Optimal Parameters Run in Previous Section
# plot_shrinkage_and_allocation(
#     log_returns_20to21,
#     alpha=2,
#     baseline_volatility=0.03,
#     baseline_correlation=0.5,
#     beta=1,
#     gamma=20,
#     delta=5,
#     window=30
# )

1st Plot: Compared to the DS-HRP framework, the shrinkage factors in HOM DS-HRP exhibit reduced stability, with noticeable fluctuations across the in-sample period. This instability suggests that the non-linear shrinkage adjustments, driven by higher-order moment parameters ($\alpha,\beta$), introduce sensitivity to market conditions and covariance matrix characteristics. The shrinkage factor remains consistently high across most periods, indicating a strong regularization effect, but the occasional sharp deviations highlight the impact of skewness and kurtosis on the shrinkage mechanism.

2nd Plot: A key improvement observed in HOM DS-HRP is the significantly enhanced stability in capital allocation across clusters. While DS-HRP exhibited frequent shifts in allocation weights, HOM DS-HRP displays smoother and more consistent allocation patterns. The refined adjustments from the higher-order moment covariance estimation reduce abrupt transitions, particularly during volatile market phases, ensuring better alignment between cluster risk profiles and capital distribution. This improved stability is primarily driven by the quasi-diagonalized covariance matrix, which exhibits reduced condition numbers across clusters. The more stable covariance structure minimizes the distortions caused by extreme market events, allowing capital to be redistributed more predictably across clusters. Notably, assets characterized by lower volatility, such as Yearn.Finance (YFI), consistently receive higher capital allocations, while higher-volatility assets like ThorChain (RUNE) and Fantom (FTM) maintain reduced allocations during risk-intensive periods.

3rd Plot: The rolling annualized volatility profiles across assets remain consistent between DS-HRP and HOM DS-HRP. Both frameworks exhibit similar volatility dynamics, with peaks and troughs aligning across the study period. This similarity indicates that while higher-order moment adjustments improve capital allocation stability, they do not significantly alter the underlying risk exposure profile derived from asset-level volatility.

### 4.8.2 Performance Evaluation (In-Sample)

In [19]:
def analyze_performance_ho(returns, rf_rate, optimal_params_dshrp, optimal_params_higher_order_dshrp, file_name):
    """
    Analyze and plot the performance of Higher-Order DS-HRP, DS-HRP, HRP, and Markowitz's mean-variance for a specific rolling year.

    Parameters:
    - returns: DataFrame of log returns for the period.
    - rf_rate: Series of risk-free rates for the period.
    - optimal_params_dshrp: Dictionary containing the optimal parameters (t, alpha, kappa_0) for DS-HRP.
    - optimal_params_higher_order_dshrp: Dictionary containing the optimal parameters (alpha, beta, baseline_volatility, baseline_correlation, gamma, delta) for Higher-Order DS-HRP.

    Returns:
    - metrics_df: DataFrame with the calculated performance metrics for each method.
    - A plot comparing the Sharpe Ratio, Volatility, and Max Drawdown of the methods.
    """

    # Higher-Order DS-HRP
    alpha_ho = optimal_params_higher_order_dshrp['alpha']
    beta_ho = optimal_params_higher_order_dshrp['beta']
    baseline_volatility_ho = optimal_params_higher_order_dshrp['baseline_volatility']
    baseline_correlation_ho = optimal_params_higher_order_dshrp['baseline_correlation']
    gamma_ho = optimal_params_higher_order_dshrp['gamma']
    delta_ho = optimal_params_higher_order_dshrp['delta']

    ho_dshrp_weights_val, ho_dshrp_sr, ho_dshrp_vol, ho_dshrp_mdd = higher_order_dshrp_weights(
        returns, rf_rate, alpha_ho, beta_ho, baseline_volatility_ho, baseline_correlation_ho, gamma_ho, delta_ho
    )
    print(f"HOM DSHRP Weights: {ho_dshrp_weights_val}")

    # DS-HRP
    t = optimal_params_dshrp['t']
    alpha = optimal_params_dshrp['alpha']
    kappa_0 = optimal_params_dshrp['kappa_0']
    dshrp_weights_val, dshrp_sr, dshrp_vol, dshrp_mdd = dshrp_weights(returns, rf_rate, t, alpha, kappa_0)
    print(f"DSHRP Weights: {dshrp_weights_val}")

    # HRP
    hrp_weights_val = hrp_weights(returns)
    hrp_sr, hrp_vol, hrp_mdd = performance(hrp_weights_val, returns, rf_rate)
    print(f"HRP Weights: {hrp_weights_val}")

    # Markowitz's
    mv_weights_val = mv_weights(returns)
    mv_sr, mv_vol, mv_mdd = performance(mv_weights_val, returns, rf_rate)
    print(f"Markowitz Weights: {mv_weights_val}")

    # Print Results
    print(f"HOM DSHRP: Sharpe Ratio = {ho_dshrp_sr:.4%}, Volatility = {ho_dshrp_vol:.4%}, MDD = {ho_dshrp_mdd:.2%}")
    print(f"DSHRP: Sharpe Ratio = {dshrp_sr:.4%}, Volatility = {dshrp_vol:.4%}, MDD = {dshrp_mdd:.2%}")
    print(f"Traditional HRP: Sharpe Ratio = {hrp_sr:.4%}, Volatility = {hrp_vol:.4%}, MDD = {hrp_mdd:.2%}")
    print(f"Markowitz: Sharpe Ratio = {mv_sr:.4%}, Volatility = {mv_vol:.4%}, MDD = {mv_mdd:.2%}")


    # Create DataFrame for Visualization
    metrics_df = pd.DataFrame({
        "Method": ["HOM DSHRP", "DSHRP", "Traditional HRP", "Markowitz"],
        "Sharpe Ratio": [ho_dshrp_sr, dshrp_sr, hrp_sr, mv_sr],
        "Volatility": [ho_dshrp_vol, dshrp_vol, hrp_vol, mv_vol],
        "Max Drawdown": [ho_dshrp_mdd, dshrp_mdd, hrp_mdd, mv_mdd]
    })

    # Plot Comparison of Metrics using Plotly Subplots
    fig = make_subplots(rows=1, cols=3, subplot_titles=("Sharpe Ratio Comparison", "Volatility Comparison", "Max Drawdown Comparison"))

    # Sharpe Ratio
    fig.add_trace(go.Bar(
        x=metrics_df["Method"],
        y=metrics_df["Sharpe Ratio"],
        name="Sharpe Ratio",
        marker_color="gold"
    ), row=1, col=1)

    # Volatility
    fig.add_trace(go.Bar(
        x=metrics_df["Method"],
        y=metrics_df["Volatility"],
        name="Volatility",
        marker_color="seagreen"
    ), row=1, col=2)

    # Max Drawdown
    fig.add_trace(go.Bar(
        x=metrics_df["Method"],
        y=metrics_df["Max Drawdown"],
        name="Max Drawdown",
        marker_color="firebrick"
    ), row=1, col=3)

    # Update Layout
    fig.update_layout(
        font=dict(size=12),
        title_x=0.5,
        height=500,
        width=1200,
        margin=dict(l=20, r=20, t=30, b=30),
        showlegend=False
    )

    # Update Axes Titles
    fig.update_xaxes(title_text="Method", row=1, col=1)
    fig.update_yaxes(title_text="Sharpe Ratio", row=1, col=1)

    fig.update_xaxes(title_text="Method", row=1, col=2)
    fig.update_yaxes(title_text="Volatility", row=1, col=2)

    fig.update_xaxes(title_text="Method", row=1, col=3)
    fig.update_yaxes(title_text="Max Drawdown", row=1, col=3)

    fig.write_image(file_name, scale=3)

    fig.show()

    return metrics_df

In [21]:
metrics_20to21 = analyze_performance_ho(log_returns_20to21, rf_rate_20to21, optimal_params_20to21, optimal_params_20to21ho, file_name="/content/drive/My Drive/data/20to21ho_sr_vol_mdd_plotly.png")

HOM DSHRP Weights: [0.1732139  0.20512283 0.19373353 0.20108858 0.22684116]
DSHRP Weights: [0.19845618 0.2405945  0.19933861 0.17624414 0.18536658]
HRP Weights: [0.17670507 0.17670507 0.21552995 0.21552995 0.21552995]
Markowitz Weights: [0.09956866 0.17715193 0.0858558  0.07416097 0.56326265]
HOM DSHRP: Sharpe Ratio = 13.1936%, Volatility = 3.6183%, MDD = 82.08%
DSHRP: Sharpe Ratio = 13.5158%, Volatility = 3.7094%, MDD = 82.79%
Traditional HRP: Sharpe Ratio = 12.9712%, Volatility = 3.6422%, MDD = 81.75%
Markowitz: Sharpe Ratio = 12.5219%, Volatility = 3.8660%, MDD = 81.29%


**Sharpe Ratio:** HOM DS-HRP achieved a Sharpe Ratio of 13.19\%, slightly lower than DS-HRP (13.52\%) but higher than both HRP (12.97\%) and Markowitz (12.52\%). The incorporation of skewness ($\alpha=2$) and kurtosis ($\beta=1$) into the covariance estimation enhanced HOM DS-HRP's ability to capture tail risks, which contributed to a competitive risk-adjusted return despite increased sensitivity in shrinkage adjustments. Although DS-HRP outperformed HOM DS-HRP in Sharpe Ratio, the HOM DS-HRP results indicate a strong balance between risk management and portfolio return optimization.

**Volatility:** The portfolio volatility under HOM DS-HRP (3.62\%) was lower than both DS-HRP (3.71\%) and Markowitz (3.87\%), while being slightly better than HRP (3.64\%). This suggests that HOM DS-HRP effectively controls portfolio-level volatility, leveraging dynamic shrinkage and higher-order moment adjustments. The lower volatility indicates that HOM DS-HRP achieved more stability during periods of market turbulence, despite its sensitivity to higher-order adjustments.

**Maximum Drawdown:** HOM DS-HRP recorded a Maximum Drawdown of 82.08\%, slightly better than DS-HRP (82.79\%) but slightly worse than HRP (81.75\%) and Markowitz (81.29\%). The moderate drawdown indicates that while HOM DS-HRP's higher-order covariance adjustments provided resilience against tail risks, there was still room for improvement in extreme market scenarios. However, the result reflects the adaptability of HOM DS-HRP in mitigating drawdowns compared to DS-HRP under similar conditions.

**Overall:** Compared to DS-HRP, HOM DS-HRP achieved lower volatility and slightly improved drawdown, despite a modestly lower Sharpe Ratio. Compared to HRP and Markowitz, HOM DS-HRP demonstrated enhanced stability in volatility control and competitive risk-adjusted returns, highlighting its robustness under higher-order adjustments. The optimized parameters ($\alpha=2$, $\beta=1$, $\sigma_0=0.03$, $\rho_0=0.5$, $\gamma=20$, $\delta=5$) allowed HOM DS-HRP to address skewness and kurtosis effectively while maintaining capital allocation consistency across clusters.

### 4.8.4 Synthetic Data Validation

In [26]:
def simulate_ar_garch_prices(N, T, clusters, cluster_size, initial_price=100, shock_intensity=0.1, random_seed=0):
    """
    Simulate synthetic asset prices using AR(1)-GARCH(1,1) process with clustered shocks.

    Parameters:
    - N: Number of assets.
    - T: Number of observations.
    - clusters: Number of clusters.
    - cluster_size: Number of assets per cluster.
    - initial_price: Starting price for each asset.
    - shock_intensity: Intensity of shocks to introduce nonstationarity.
    - random_seed: Seed for reproducibility.

    Returns:
    - DataFrame of simulated prices.
    """
    np.random.seed(random_seed)  # Global seed for reproducibility
    prices = np.zeros((T, N))
    prices[0, :] = initial_price  # Set initial prices

    omega = 0.1  # GARCH(1,1) omega
    alpha = 0.15  # GARCH(1,1) alpha
    beta = 0.7  # GARCH(1,1) beta
    phi = 0.2  # AR(1) coefficient
    mu = 0.0  # AR(1) mean term

    asset_index = 0
    for i in range(clusters):
        for j in range(cluster_size):
            current_seed = random_seed + asset_index  # Unique seed per asset
            np.random.seed(current_seed)

            # Initialize variables
            returns = np.zeros(T)
            volatilities = np.zeros(T)
            shocks = np.random.randn(T)  # Pre-generated random shocks

            # Initial volatility
            volatilities[0] = np.sqrt(omega / (1 - alpha - beta))

            for t in range(1, T):
                # Update volatility (GARCH(1,1))
                volatilities[t] = np.sqrt(omega + alpha * (returns[t-1] ** 2) + beta * (volatilities[t-1] ** 2))

                # Generate return (AR(1) + shock)
                returns[t] = mu + phi * returns[t-1] + volatilities[t] * shocks[t]

            # Ensure deterministic shocks for the last cluster
            if i == clusters - 1:
                np.random.seed(current_seed + 1000)  # Separate seed for shocks
                shocks_cluster = shock_intensity * np.random.randn(T)
                returns += shocks_cluster

            # Convert returns to prices
            prices[:, asset_index] = initial_price * np.exp(np.cumsum(returns))
            asset_index += 1

    return pd.DataFrame(prices, columns=[f"Asset_{i+1}" for i in range(N)])

#### Random Seed = 0

In [ ]:
# Reproducibility Test
random_seed = 0

# Parameters
N = 5  # Number of assets
T = 500  # Number of observations
clusters = 5  # Number of clusters
cluster_size = N // clusters
shock_intensity = 0.1  # Intensity of shocks to introduce nonstationarity
initial_price = 100  # Starting price for each asset

# Generate synthetic prices
synthetic_prices = simulate_ar_garch_prices(
    N, T, clusters, cluster_size, initial_price, shock_intensity, random_seed
)

# Calculate Log Returns from Prices
synthetic_log_returns = np.log(synthetic_prices / synthetic_prices.shift(1)).dropna()

# Display results
print(synthetic_prices.head())
print(synthetic_log_returns.head())

# Simulate Synthetic Risk-Free Rates
np.random.seed(random_seed)
rf_base_rate = 0.03  # Base rate of 3%
rf_volatility = 0.002  # Small volatility to simulate rate changes
synthetic_rf_rate = pd.Series(rf_base_rate + rf_volatility * np.random.randn(T), name="RiskFreeRate")

        Asset_1     Asset_2     Asset_3     Asset_4     Asset_5
0    100.000000  100.000000  100.000000  100.000000  106.124241
1    135.151665   63.095957   95.852836  138.901197  160.970490
2    288.816585   39.196671   21.082847  158.949654   78.199054
3   1754.624414   16.638172   67.033687   46.030080  114.971557
4  15821.282482   26.832188   16.090404   28.708196   85.795174
    Asset_1   Asset_2   Asset_3   Asset_4   Asset_5
1  0.301227 -0.460513 -0.042356  0.328593  0.416611
2  0.759394 -0.476065 -1.514354  0.134825 -0.721964
3  1.804218 -0.856892  1.156736 -1.239292  0.385427
4  2.199101  0.477903 -1.426972 -0.472112 -0.292722
5 -0.758537 -1.577247 -1.128797 -0.367136 -1.183274


##### Optimal Parameters

In [ ]:
optimal_params_syn, optimal_weights_syn, optimal_sr_syn, optimal_vol_syn, optimal_mdd_syn = find_optimal_params(
    synthetic_log_returns,
    synthetic_rf_rate,
    param_grid_dshrp
)


Optimal DSHRP Parameters:
t: 0.1, alpha: 0.9, kappa_0: 39
Optimal Portfolio Weights: [0.20632782 0.20722117 0.19258253 0.19438011 0.19948837]
Optimal Sharpe Ratio: 4.21%
Portfolio Volatility: 35.87%
Maximum Drawdown: 101.13%


In [ ]:
# optimal_params_syn = {'t': 0.1, 'alpha': 0.8, 'kappa_0': 45}

In [ ]:
optimal_params_syn_ho, optimal_weights_syn_ho, optimal_sr_syn_ho, optimal_vol_syn_ho, optimal_mdd_syn_ho = find_optimal_params_higher_order(
    synthetic_log_returns,
    synthetic_rf_rate,
    param_grid_hom_dshrp,
    "/content/drive/My Drive/data/sharpe_ratio_optimization_syn_plotly.png"
)


Optimal Higher-Order DSHRP Parameters:
alpha: 0.0, beta: 1, baseline_volatility: 0.32999999999999996, baseline_correlation: 0.1, gamma: 5, delta: 5
Optimal Portfolio Weights: [0.16156134 0.22717518 0.18082078 0.28352255 0.14692015]
Optimal Sharpe Ratio: 3.74%
Portfolio Volatility: 37.04%
Maximum Drawdown: 100.00%


In [ ]:
# optimal_params_syn_ho = {'alpha': 0.1, 'beta': 0.30000000000000004, 'baseline_volatility': 0.3, 'baseline_correlation': 0.1, 'gamma': 5, 'delta': 5}

##### Performance

In [ ]:
metrics_df = analyze_performance_ho(synthetic_log_returns, synthetic_rf_rate, optimal_params_syn, optimal_params_syn_ho, file_name="/content/drive/My Drive/data/synthetic500garch_performance_plotly.png")

#### Random Seed = 42

In [27]:
# Reproducibility Test
random_seed = 42

# Parameters
N = 5  # Number of assets
T = 500  # Number of observations
clusters = 5  # Number of clusters
cluster_size = N // clusters
shock_intensity = 0.1  # Intensity of shocks to introduce nonstationarity
initial_price = 100  # Starting price for each asset

# Generate synthetic prices
synthetic_prices = simulate_ar_garch_prices(
    N, T, clusters, cluster_size, initial_price, shock_intensity, random_seed
)

# Calculate Log Returns from Prices
synthetic_log_returns = np.log(synthetic_prices / synthetic_prices.shift(1)).dropna()

# Display results
print(synthetic_prices.head())
print(synthetic_log_returns.head())

# Simulate Synthetic Risk-Free Rates
np.random.seed(random_seed)
rf_base_rate = 0.03  # Base rate of 3%
rf_volatility = 0.002  # Small volatility to simulate rate changes
synthetic_rf_rate = pd.Series(rf_base_rate + rf_volatility * np.random.randn(T), name="RiskFreeRate")

      Asset_1     Asset_2     Asset_3     Asset_4     Asset_5
0  100.000000  100.000000  100.000000  100.000000  103.769624
1   90.115177   50.465514  269.370561  121.648196  255.972503
2  139.417657   33.100513  892.707196   95.606278  593.358357
3  435.738129   20.659770  278.424457   79.315628  393.916921
4  454.526544   34.484536   57.433555   33.268818  427.845700
    Asset_1   Asset_2   Asset_3   Asset_4   Asset_5
1 -0.104082 -0.683880  0.990918  0.195963  0.902897
2  0.436386 -0.421741  1.198171 -0.240895  0.840729
3  1.139567 -0.471360 -1.165112 -0.186803 -0.409659
4  0.042215  0.512323 -1.578518 -0.868815  0.082623
5 -0.163697 -0.186404 -2.082882 -2.032822 -0.141946


##### Optimal Parameters

In [ ]:
optimal_params_syn42, optimal_weights_syn42, optimal_sr_syn42, optimal_vol_syn42, optimal_mdd_syn42 = find_optimal_params(
    synthetic_log_returns,
    synthetic_rf_rate,
    param_grid_dshrp
)


Optimal DSHRP Parameters:
t: 0.1, alpha: 0.7000000000000001, kappa_0: 48
Optimal Portfolio Weights: [0.19408612 0.20465269 0.20737157 0.20218542 0.1917042 ]
Optimal Sharpe Ratio: 2.29%
Portfolio Volatility: 35.17%
Maximum Drawdown: 100.00%


In [31]:
# optimal_params_syn42 = {'t': 0.1, 'alpha': 0.7000000000000001, 'kappa_0': 48}

In [ ]:
optimal_params_syn_ho42, optimal_weights_syn_ho42, optimal_sr_syn_ho42, optimal_vol_syn_ho42, optimal_mdd_syn_ho42 = find_optimal_params_higher_order(
    synthetic_log_returns,
    synthetic_rf_rate,
    param_grid_hom_dshrp,
    "/content/drive/My Drive/data/sharpe_ratio_optimization_syn_plotly42.png"
)


Optimal Higher-Order DSHRP Parameters:
alpha: 2.0, beta: 1, baseline_volatility: 0.03, baseline_correlation: 0.5, gamma: 20, delta: 20
Optimal Portfolio Weights: [0.1720195  0.21094424 0.27361627 0.17770895 0.16571104]
Optimal Sharpe Ratio: 3.19%
Portfolio Volatility: 35.60%
Maximum Drawdown: 103.46%


In [32]:
# optimal_params_syn_ho42 = {'alpha': 2, 'beta': 1, 'baseline_volatility': 0.03, 'baseline_correlation': 0.5, 'gamma': 20, 'delta': 20}

The Sharpe ratio optimization plot for the synthetic dataset reveals a clear progression through distinct phases. In the initial phase (0–1000 iterations), improvements are minimal, indicating limited benefits from early parameter combinations. During the refinement phase (1000–4000 iterations), the Sharpe ratio demonstrates steady incremental gains as the grid search explores more effective parameter configurations. In the convergence phase (4000–7000 iterations), the optimization process stabilizes, with occasional sharp jumps reflecting the discovery of near-optimal parameter sets. By the final optimization stage (~6500 iterations), the Sharpe ratio reaches its peak, suggesting an optimal balance between skewness, kurtosis, volatility sensitivity, and correlation dynamics. Compared to the in-sample Sharpe ratio optimization plot, this synthetic plot appears cleaner, with smoother transitions and fewer abrupt fluctuations, reflecting the controlled and less noisy nature of the synthetic data environment.

##### Performance

In [33]:
metrics_df42 = analyze_performance_ho(synthetic_log_returns, synthetic_rf_rate, optimal_params_syn42, optimal_params_syn_ho42, file_name="/content/drive/My Drive/data/synthetic500garch_performance_plotly42.png")

HOM DSHRP Weights: [0.1720195  0.21094424 0.27361627 0.17770895 0.16571104]
DSHRP Weights: [0.19408612 0.20465269 0.20737157 0.20218542 0.1917042 ]
HRP Weights: [0.18433548 0.18433548 0.21044302 0.21044302 0.21044302]
Markowitz Weights: [0.23386952 0.23203867 0.18494969 0.18249646 0.16664567]
HOM DSHRP: Sharpe Ratio = 3.1936%, Volatility = 35.5987%, MDD = 103.46%
DSHRP: Sharpe Ratio = 2.2911%, Volatility = 35.1733%, MDD = 100.00%
Traditional HRP: Sharpe Ratio = 2.0931%, Volatility = 35.3064%, MDD = 100.00%
Markowitz: Sharpe Ratio = 2.2901%, Volatility = 35.2082%, MDD = 100.01%


**Sharpe Ratio:** HOM DS-HRP achieved the highest Sharpe ratio (3.19\%), significantly outperforming DS-HRP (2.29\%), HRP (2.09\%), and Markowitz (2.29\%). This demonstrates the method's ability to deliver superior risk-adjusted returns, even in a highly volatile synthetic environment.

**Volatility:** Portfolio volatility for HOM DS-HRP (35.60\%) was slightly higher than DS-HRP (35.17\%) and HRP (35.31\%) but marginally lower than Markowitz (35.21\%). This indicates a balanced risk exposure profile under the HOM DS-HRP framework.

**Maximum Drawdown:** The maximum drawdown for HOM DS-HRP (103.46\%) was notably higher than all other methods, suggesting that while the method excels in risk-adjusted returns, it carries elevated tail risk in extreme scenarios.

### 4.8.5 Sensitivity and Robustness Analyses (In-Sample)

#### DS-HRP

In [ ]:
def plot_dshrp_sensitivity(returns, rf_rate, t_range, alpha_range, kappa_0_range):
    """
    Plot Sharpe Ratio (SR), Volatility (Vol), and Max Drawdown (MDD) against parameters for DS-HRP.

    Parameters:
    - returns: DataFrame of asset returns.
    - rf_rate: Series of risk-free rates.
    - t_range: Range of `t` parameter values.
    - alpha_range: Range of `alpha` parameter values.
    - kappa_0_range: Range of `kappa_0` parameter values.
    """
    # Parameter names and their ranges
    params = [
        ("t", t_range),
        ("alpha", alpha_range),
        ("kappa_0", kappa_0_range)
    ]

    # Prepare to store results
    results = {param[0]: {"sr": [], "vol": [], "mdd": []} for param in params}

    # Compute outputs for each parameter
    for name, param_range in params:
        for value in param_range:
            # Set parameter values dynamically
            kwargs = {
                "t": 0.5,
                "alpha": 0.5,
                "kappa_0": 20
            }
            kwargs[name] = value  # Override the specific parameter

            # Compute metrics
            _, sr, vol, mdd = dshrp_weights(returns, rf_rate, **kwargs)
            results[name]["sr"].append(sr)
            results[name]["vol"].append(vol)
            results[name]["mdd"].append(mdd)

    # Define consistent colors for metrics
    metric_colors = {
        "sr": "#1f77b4",  # Blue for Sharpe Ratio
        "vol": "#ff7f0e",  # Orange for Volatility
        "mdd": "#2ca02c"  # Green for Max Drawdown
    }

    # Create subplots with Plotly
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=["Metrics vs t", "Metrics vs alpha", "Metrics vs kappa_0"]
    )

    # Plot results
    for i, (name, param_range) in enumerate(params):
        # Sharpe Ratio
        fig.add_trace(
            go.Scatter(
                x=param_range, y=results[name]["sr"],
                mode='lines+markers',
                name='Sharpe Ratio' if i == 0 else None,
                marker=dict(symbol='circle', size=8),
                line=dict(color=metric_colors["sr"], dash='solid'),
                showlegend=(i == 0)
            ),
            row=1, col=i+1
        )
        # Volatility
        fig.add_trace(
            go.Scatter(
                x=param_range, y=results[name]["vol"],
                mode='lines+markers',
                name='Volatility' if i == 0 else None,
                marker=dict(symbol='square', size=8),
                line=dict(color=metric_colors["vol"], dash='solid'),
                showlegend=(i == 0)
            ),
            row=1, col=i+1
        )
        # Max Drawdown
        fig.add_trace(
            go.Scatter(
                x=param_range, y=results[name]["mdd"],
                mode='lines+markers',
                name='Max Drawdown' if i == 0 else None,
                marker=dict(symbol='triangle-up', size=8),
                line=dict(color=metric_colors["mdd"], dash='solid'),
                showlegend=(i == 0)
            ),
            row=1, col=i+1
        )

        # Update axis titles
        fig.update_xaxes(title_text=name, row=1, col=i+1)
        fig.update_yaxes(title_text="Metric Values", row=1, col=i+1)

    # Update layout
    fig.update_layout(
        title={
            'text': "Sensitivity Analysis for DS-HRP",
            'x': 0.5,
            'xanchor': 'center',
            'y': 0.98,
            'yanchor': 'top'
        },
        font=dict(size=12),
        height=400,
        width=1200,
        showlegend=True,
        legend=dict(
            font=dict(size=12),
            orientation="h",
            yanchor="bottom",
            y=-0.3,
            xanchor="center",
            x=0.5
        ),
        margin=dict(l=20, r=20, t=50, b=20)
    )

    # Save and display the plot
    fig.write_image("/content/drive/My Drive/data/20to21_sensitivity_analysis_plotly.png", scale=3)
    fig.show()

# Parameter Range
t_range = np.linspace(0, 2, 20)
alpha_range = np.linspace(0, 2, 20)
kappa_0_range = np.linspace(0, 50, 20)

plot_dshrp_sensitivity(log_returns_20to21, rf_rate_20to21, t_range, alpha_range, kappa_0_range)

The figure shows the sensitivity results for DS-HRP. While DS-HRP demonstrates commendable robustness, it exhibits a more pronounced sensitivity to shrinkage intensity ($t$) and cluster adjustment thresholds ($\kappa_0$). These sensitivities result in localized dips in the Sharpe Ratio and slight increases in volatility when parameter values deviate from optimal settings.

#### HOM DS-HRP

In [ ]:
def plot_higher_order_dshrp_metrics_for_parameters(data, rf_rate, param_ranges):
    """
    Plot Sharpe Ratio (SR), Volatility (Vol), and Max Drawdown (MDD) against each parameter.

    Parameters:
    - data: A Pandas DataFrame of asset returns.
    - rf_rate: Risk-free rate for Sharpe ratio calculation.
    - param_ranges: Dictionary containing parameter names and their ranges as lists.
    """
    # Parameter names and their ranges
    params = [
        ("Alpha", param_ranges['alpha']),
        ("Beta", param_ranges['beta']),
        ("Baseline Volatility", param_ranges['baseline_volatility']),
        ("Baseline Correlation", param_ranges['baseline_correlation']),
        ("Gamma", param_ranges['gamma']),
        ("Delta", param_ranges['delta']),
    ]

    # Prepare to store results
    results = {param[0]: {"sr": [], "vol": [], "mdd": []} for param in params}

    # Compute outputs for each parameter
    for name, param_range in params:
        for value in param_range:
            # Set parameter values dynamically
            kwargs = {
                "alpha": 0.5,
                "beta": 5,
                "baseline_volatility": 0.1,
                "baseline_correlation": 0.3,
                "gamma": 10,
                "delta": 10,
            }
            kwargs[name.lower().replace(" ", "_")] = value  # Override the specific parameter

            # Compute metrics
            _, sr, vol, mdd = higher_order_dshrp_weights(data, rf_rate, **kwargs)
            results[name]["sr"].append(sr)
            results[name]["vol"].append(vol)
            results[name]["mdd"].append(mdd)

    # Define consistent colors for metrics
    metric_colors = {
        "sr": "#1f77b4",  # Blue for Sharpe Ratio
        "vol": "#ff7f0e",  # Orange for Volatility
        "mdd": "#2ca02c"  # Green for Max Drawdown
    }

    # Create subplots with Plotly
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[f"Metrics vs {name}" for name, _ in params],
        vertical_spacing=0.1,
        horizontal_spacing=0.05
    )

    # Plot results
    for i, (name, param_range) in enumerate(params):
        row, col = divmod(i, 2)
        row += 1
        col += 1

        # Sharpe Ratio
        fig.add_trace(
            go.Scatter(
                x=param_range,
                y=results[name]["sr"],
                mode='lines+markers',
                name='Sharpe Ratio' if i == 0 else None,
                marker=dict(symbol='circle', size=6),
                line=dict(color=metric_colors["sr"], dash='solid'),
                showlegend=(i == 0)
            ),
            row=row, col=col
        )

        # Volatility
        fig.add_trace(
            go.Scatter(
                x=param_range,
                y=results[name]["vol"],
                mode='lines+markers',
                name='Volatility' if i == 0 else None,
                marker=dict(symbol='square', size=6),
                line=dict(color=metric_colors["vol"], dash='solid'),
                showlegend=(i == 0)
            ),
            row=row, col=col
        )

        # Max Drawdown
        fig.add_trace(
            go.Scatter(
                x=param_range,
                y=results[name]["mdd"],
                mode='lines+markers',
                name='Max Drawdown' if i == 0 else None,
                marker=dict(symbol='triangle-up', size=6),
                line=dict(color=metric_colors["mdd"], dash='solid'),
                showlegend=(i == 0)
            ),
            row=row, col=col
        )

        # Update axis titles for clarity
        fig.update_xaxes(title_text=name, row=row, col=col)
        fig.update_yaxes(title_text="Metric Values", row=row, col=col)

    # Update layout
    fig.update_layout(
        title={
            'text': "Sensitivity Analysis for Higher-Order DSHRP",
            'x': 0.5,
            'xanchor': 'center',
            'y': 0.97,
            'yanchor': 'top'
        },
        font=dict(size=12),
        height=900,
        width=1200,
        showlegend=True,
        legend=dict(
            font=dict(size=12),
            orientation="h",
            yanchor="bottom",
            y=-0.1,
            xanchor="center",
            x=0.5
        ),
        margin=dict(l=20, r=20, t=80, b=20)
    )

    # Save and display the plot
    fig.write_image("/content/drive/My Drive/data/20to21ho_sensitivity_analysis_plotly.png", scale=3)
    fig.show()


# Parameter Range
param_ranges = {
    "alpha": np.linspace(0, 2, 20),
    "beta": np.linspace(0, 10, 20),
    "baseline_volatility": np.linspace(0, 1, 20),
    "baseline_correlation": np.linspace(-1, 1, 20),
    "gamma": np.linspace(0, 20, 20),
    "delta": np.linspace(0, 20, 20),
}

plot_higher_order_dshrp_metrics_for_parameters(
    log_returns_20to21,
    rf_rate_20to21,
    param_ranges
)

The figure illustrates the results of these sensitivity tests. The analysis reveals that HOM DS-HRP exhibits remarkable resilience across diverse parameter configurations, with minimal deviations in key performance metrics. The Sharpe Ratio, Volatility, and Maximum Drawdown remain largely stable across the parameter ranges tested, demonstrating the framework's ability to adapt effectively to varying skewness, kurtosis, and baseline risk conditions. While slight variations emerge at extreme parameter values, they do not significantly disrupt overall portfolio performance, highlighting HOM DS-HRP's structural robustness.

# Simulation Results

## Original Simulation Results


In [ ]:
# Generate Synthetic Asset Prices with Clustered Correlations
def simulate_clustered_prices(N, T, clusters, cluster_size, initial_price, shock_intensity):
    """Simulate synthetic asset prices with clustered correlation structure and shocks."""
    prices = np.zeros((T, N))
    prices[0, :] = initial_price  # Set initial price

    for i in range(clusters):
        # Generate random data to compute a valid correlation matrix
        random_data = np.random.rand(T, cluster_size)
        cluster_corr = np.corrcoef(random_data, rowvar=False)

        # Generate cluster price changes using the valid correlation matrix
        cluster_returns = np.random.multivariate_normal(mean=np.zeros(cluster_size), cov=cluster_corr, size=T)

        # Introduce shocks to individual assets and clusters
        if i == clusters - 1:
            cluster_returns += shock_intensity * np.random.randn(T, cluster_size)

        # Convert returns to prices
        start_idx = i * cluster_size
        end_idx = start_idx + cluster_size
        for j in range(cluster_size):
            prices[:, start_idx + j] = initial_price * np.exp(np.cumsum(cluster_returns[:, j]))  # Cumulative product for price path

    return pd.DataFrame(prices, columns=[f"Asset_{i+1}" for i in range(N)])

In [ ]:
def plot_synthetic_assets_and_returns(N, T, clusters, initial_price, shock_intensity, file_name):

    cluster_size = N // clusters

    # Generate synthetic prices
    synthetic_prices = simulate_clustered_prices(N, T, clusters, cluster_size, initial_price, shock_intensity)

    # Calculate Log Returns from Prices
    synthetic_log_returns = np.log(synthetic_prices / synthetic_prices.shift(1)).dropna()

    # Simulate Synthetic Risk-Free Rates
    rf_base_rate = 0.03  # Base rate of 3%
    rf_volatility = 0.002  # Small volatility to simulate rate changes
    rf_rate = pd.Series(rf_base_rate + rf_volatility * np.random.randn(T), name="RiskFreeRate")

    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("Synthetic Asset Prices of Sample Assets", "Synthetic Log Returns of Sample Assets")
    )

    # Plot synthetic prices for a sample of assets (first 5 assets)
    for i in range(min(len(synthetic_prices.columns), 5)):
        fig.add_trace(
            go.Scatter(x=synthetic_prices.index, y=synthetic_prices.iloc[:, i],
                       mode='lines', name=f"Asset_{i+1} Price"),
            row=1, col=1
        )

    # Plot synthetic log returns for a sample of assets (first 5 assets)
    for i in range(min(len(synthetic_log_returns.columns), 5)):
        fig.add_trace(
            go.Scatter(x=synthetic_log_returns.index, y=synthetic_log_returns.iloc[:, i],
                       mode='lines', name=f"Asset_{i+1} Log Return"),
            row=1, col=2
        )

    fig.update_layout(height=500, width=1200, margin=dict(l=50, r=50, t=50, b=50))
    fig.update_xaxes(title_text="Time", row=1, col=1)
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_xaxes(title_text="Time", row=1, col=2)
    fig.update_yaxes(title_text="Log Return", row=1, col=2)

    fig.write_image(file_name, scale=3)

    fig.show()

    return synthetic_log_returns, rf_rate

### T = 500

#### Synthetic Asset Prices and Log Returns

In [ ]:
# Reproducibility
np.random.seed(0)
simulated_log_returns500, simulated_rf_rate500 = plot_synthetic_assets_and_returns(N=50, T=500, clusters=5, initial_price=100, shock_intensity=0.1, file_name="/content/drive/My Drive/data/synthetic500_returns_plotly.png")

Synthetic asset price trajectories and log returns of sample assets with 500 observations.

#### Optimal Parameters and Performacne

In [ ]:
optimal_params500, optimal_weights500, optimal_sr500, optimal_vol500, optimal_mdd500 = find_optimal_params(simulated_log_returns500, simulated_rf_rate500, param_grid_dshrp)


Optimal DSHRP Parameters:
t: 0.6500000000000001, alpha: 0.8, kappa_0: 44
Optimal Portfolio Weights: [0.         0.         0.         0.0421939  0.04024184 0.04026229
 0.         0.         0.03943493 0.         0.03956716 0.03883272
 0.         0.         0.         0.         0.         0.04051179
 0.03887869 0.         0.04163083 0.04145986 0.0418283  0.0403146
 0.         0.03798816 0.         0.         0.         0.03926081
 0.         0.         0.03888265 0.         0.         0.03963315
 0.         0.04096058 0.03943258 0.03901385 0.04104241 0.
 0.         0.         0.03847201 0.04022021 0.03911256 0.04045891
 0.04036522 0.        ]
Optimal Sharpe Ratio: 2.35%
Portfolio Volatility: 19.68%
Maximum Drawdown: 100.00%


In [ ]:
simulated_metrics500 = analyze_performance(simulated_log_returns500, simulated_rf_rate500, optimal_params500, file_name="/content/drive/My Drive/data/synthetic500_performance_plotly.png")
print("\nPerformance Metrics for Synthetic Data:\n", simulated_metrics500)

DSHRP Weights: [0.         0.         0.         0.0421939  0.04024184 0.04026229
 0.         0.         0.03943493 0.         0.03956716 0.03883272
 0.         0.         0.         0.         0.         0.04051179
 0.03887869 0.         0.04163083 0.04145986 0.0418283  0.0403146
 0.         0.03798816 0.         0.         0.         0.03926081
 0.         0.         0.03888265 0.         0.         0.03963315
 0.         0.04096058 0.03943258 0.03901385 0.04104241 0.
 0.         0.         0.03847201 0.04022021 0.03911256 0.04045891
 0.04036522 0.        ]
HRP Weights: [0.01959304 0.01959304 0.01959304 0.01959304 0.01959304 0.01959304
 0.01959304 0.01959304 0.01959304 0.01959304 0.01959304 0.01959304
 0.01959304 0.01959304 0.01959304 0.01959304 0.01959304 0.01959304
 0.01959304 0.01959304 0.01959304 0.01959304 0.01959304 0.01959304
 0.01959304 0.02040696 0.02040696 0.02040696 0.02040696 0.02040696
 0.02040696 0.02040696 0.02040696 0.02040696 0.02040696 0.02040696
 0.02040696 0.02040


Performance Metrics for Synthetic Data:
             Method  Sharpe Ratio  Volatility  Max Drawdown
0            DSHRP      0.023469    0.196789      0.999971
1  Traditional HRP     -0.004963    0.139980      0.997960
2        Markowitz     -0.002021    0.150454      0.993276


**Sharpe Ratio:** The DS-HRP approach achieved a Sharpe Ratio of 2.35\%, significantly outperforming both HRP (-0.50\%) and Markowitz (-0.20\%). This superior risk-adjusted return highlights the robustness of the DS-HRP framework in handling clustered volatility and correlation patterns within a synthetic dataset.

**Volatility:** DS-HRP exhibited a portfolio volatility of 19.68\%, which is higher than HRP (14.00\%) and Markowitz (15.05\%). While higher volatility may suggest increased sensitivity to market fluctuations, it aligns with the risk-return optimization objectives of the DS-HRP approach.

**Maximum Drawdown:** The maximum drawdown for DS-HRP reached 99.99\%, closely matching HRP (99.80\%) and Markowitz (99.33\%). These substantial drawdowns indicate that even advanced allocation frameworks remain susceptible to extreme market conditions in synthetic environments.

**Overall:** The DS-HRP approach demonstrated superior performance in terms of risk-adjusted returns, albeit with increased portfolio volatility and drawdown levels. This suggests that DS-HRP is particularly effective in scenarios requiring dynamic covariance matrix adjustments to optimize returns under complex synthetic market conditions.

### T = 1000

#### Synthetic Asset Prices and Log Returns

In [ ]:
# Reproducibility
np.random.seed(0)
simulated_log_returns1000, simulated_rf_rate1000 = plot_synthetic_assets_and_returns(N=50, T=1000, clusters=5, initial_price=100, shock_intensity=0.1, file_name="/content/drive/My Drive/data/synthetic1000_returns_plotly.png")

Synthetic asset price trajectories and log returns of sample assets with 1000 observations.

#### Optimal Parameters and Performance

In [ ]:
optimal_params1000, optimal_weights1000, optimal_sr1000, optimal_vol1000, optimal_mdd1000 = find_optimal_params(simulated_log_returns1000, simulated_rf_rate1000, param_grid_dshrp)


Optimal DSHRP Parameters:
t: 0.1, alpha: 0.8, kappa_0: 45
Optimal Portfolio Weights: [0.02036235 0.01986519 0.0201718  0.02094096 0.01905288 0.02028256
 0.02033208 0.02004818 0.0202202  0.01989099 0.0197039  0.02071839
 0.02034308 0.01970529 0.01898095 0.02028067 0.01973681 0.02096724
 0.01985032 0.02062079 0.01989506 0.02033912 0.02077803 0.02005299
 0.02006575 0.01968238 0.0199287  0.01956585 0.01959731 0.01929155
 0.01963243 0.0201614  0.02032112 0.01979179 0.02007453 0.01967167
 0.0198017  0.02039642 0.02008916 0.01955871 0.01983647 0.02005961
 0.02022766 0.01995169 0.01930961 0.02012661 0.01992758 0.02009581
 0.01999801 0.01969669]
Optimal Sharpe Ratio: 0.72%
Portfolio Volatility: 14.08%
Maximum Drawdown: 100.00%


In [ ]:
simulated_metrics1000 = analyze_performance(simulated_log_returns1000, simulated_rf_rate1000, optimal_params1000, file_name="/content/drive/My Drive/data/synthetic1000_performance_plotly.png")
print("\nPerformance Metrics for Synthetic Data:\n", simulated_metrics1000)

DSHRP Weights: [0.02036235 0.01986519 0.0201718  0.02094096 0.01905288 0.02028256
 0.02033208 0.02004818 0.0202202  0.01989099 0.0197039  0.02071839
 0.02034308 0.01970529 0.01898095 0.02028067 0.01973681 0.02096724
 0.01985032 0.02062079 0.01989506 0.02033912 0.02077803 0.02005299
 0.02006575 0.01968238 0.0199287  0.01956585 0.01959731 0.01929155
 0.01963243 0.0201614  0.02032112 0.01979179 0.02007453 0.01967167
 0.0198017  0.02039642 0.02008916 0.01955871 0.01983647 0.02005961
 0.02022766 0.01995169 0.01930961 0.02012661 0.01992758 0.02009581
 0.01999801 0.01969669]
HRP Weights: [0.02065368 0.02065368 0.02065368 0.02065368 0.02065368 0.02065368
 0.02065368 0.02065368 0.02065368 0.02065368 0.02065368 0.02065368
 0.02065368 0.02065368 0.02065368 0.02065368 0.02065368 0.02065368
 0.02065368 0.02065368 0.02065368 0.02065368 0.02065368 0.02065368
 0.02065368 0.01934632 0.01934632 0.01934632 0.01934632 0.01934632
 0.01934632 0.01934632 0.01934632 0.01934632 0.01934632 0.01934632
 0.0193463


Performance Metrics for Synthetic Data:
             Method  Sharpe Ratio  Volatility  Max Drawdown
0            DSHRP      0.007163    0.140758      0.999999
1  Traditional HRP      0.005641    0.140818      0.999999
2        Markowitz      0.007216    0.145820      0.999997


**Sharpe Ratio:** The DS-HRP framework achieved a Sharpe Ratio of 0.72\%, outperforming HRP (0.56\%) and closely aligning with Markowitz (0.72\%). While DS-HRP maintains a slight edge over HRP, its performance converges with Markowitz as the time horizon extends, suggesting diminished advantages in risk-adjusted returns over longer periods.

**Volatility:** DS-HRP exhibited a portfolio volatility of 14.08\%, closely matching HRP (14.08\%) but remaining lower than Markowitz (14.58\%). This highlights DS-HRP's capacity to stabilize portfolio volatility under extended time horizons while maintaining competitive returns.

**Maximum Drawdown:** The maximum drawdown for all three methods reached approximately 100\%. This outcome reflects the extreme nature of synthetic shocks applied in the simulation and suggests a shared vulnerability across all methods when faced with prolonged periods of extreme market stress.

**Overall:** DS-HRP maintains robust risk-adjusted returns and controlled volatility across the extended simulation horizon. However, its advantage over traditional HRP and Markowitz diminishes as the time horizon increases, highlighting a potential area for refinement in long-term stability and drawdown mitigation.

## Further Enhancement Simulation Results

### Simulation Design and Data Generation Process

In [ ]:
def simulate_multivariate_garch_with_higher_moments(N, T, nu=8):
    """
    Simulate asset returns using a multivariate GARCH model with higher-order moments.

    Parameters:
    - N: Number of assets
    - T: Number of time periods
    - nu: Degrees of freedom for the Student's t-distribution

    Returns:
    - synthetic_prices: DataFrame of simulated asset prices
    - synthetic_log_returns: DataFrame of simulated asset log returns
    """
    log_returns = np.zeros((T, N))
    log_prices = np.zeros((T, N))
    log_prices[0, :] = np.log(100)  # Start prices at 100 in log-scale

    H = np.eye(N)  # Initial covariance matrix

    for t in range(1, T):
        try:
            # Generate random innovations from multivariate t-distribution
            z = np.random.standard_t(nu, N)
            eps = np.linalg.cholesky(H).dot(z)

            # Cap extreme innovations to prevent overflow
            eps = np.clip(eps, -1, 1)

            # Update log-returns
            log_returns[t, :] = eps
            log_prices[t, :] = log_prices[t-1, :] + eps  # Logarithmic price changes

            # Update covariance matrix using DCC-GARCH dynamics
            new_H = 0.9 * H + 0.1 * np.outer(eps, eps)

            # Ensure Positive Semi-Definiteness using Eigenvalue Clipping
            eigvals, eigvecs = np.linalg.eigh(new_H)
            eigvals = np.clip(eigvals, a_min=1e-8, a_max=None)  # Replace negative eigenvalues
            H = eigvecs @ np.diag(eigvals) @ eigvecs.T  # Reconstruct matrix

        except np.linalg.LinAlgError:
            print(f"Numerical instability at time {t}, resetting covariance matrix.")
            H = np.eye(N)  # Reset to identity if instability occurs

    synthetic_log_returns = pd.DataFrame(log_returns, columns=[f'Asset_{i+1}' for i in range(N)])
    synthetic_prices = pd.DataFrame(np.exp(log_prices), columns=[f'Asset_{i+1}' for i in range(N)])

    return synthetic_prices, synthetic_log_returns


### Hyperparameter Calibration and Evaluation Metrics

In [ ]:
def evaluate_portfolio_metrics(weights, returns, true_cov_matrix, expected_returns, previous_weights=None, rebalance_period=1):
    """
    Evaluate portfolio performance based on given metrics.

    Parameters:
    - weights: Array or DataFrame of portfolio weights.
    - returns: DataFrame of asset returns.
    - true_cov_matrix: True covariance matrix of asset returns.
    - expected_returns: Expected returns vector.
    - previous_weights: Previous period's weights for turnover calculation.
    - rebalance_period: Frequency of rebalancing for turnover calculation.

    Returns:
    - metrics: Dictionary containing Out-of-Sample Variance, Sharpe Ratio, Maximum Drawdown, and Portfolio Turnover.
    """

    # Out-of-Sample Variance
    portfolio_variance = np.dot(weights.T, np.dot(true_cov_matrix, weights))
    out_of_sample_variance = portfolio_variance

    # Sharpe Ratio
    portfolio_return = np.dot(weights.T, expected_returns)
    portfolio_std = np.sqrt(portfolio_variance)
    sharpe_ratio = portfolio_return / portfolio_std if portfolio_std != 0 else 0

    # Maximum Drawdown
    cumulative_returns = (1 + (returns @ weights)).cumprod()
    running_max = cumulative_returns.cummax()
    drawdown = (running_max - cumulative_returns) / running_max
    max_drawdown = drawdown.max()

    # Portfolio Turnover
    if previous_weights is not None:
        turnover = np.sum(np.abs(weights - previous_weights)) / 2
    else:
        # Assume an initial investment distribution
        turnover = np.sum(np.abs(weights - (1 / len(weights)))) / 2

    metrics = {
        "Out-of-Sample Variance": out_of_sample_variance,
        "Sharpe Ratio": sharpe_ratio,
        "Maximum Drawdown": max_drawdown,
        "Portfolio Turnover": turnover
    }

    return metrics

In [ ]:
def dshrp_weights_sim(returns, rf_rate, t, alpha, kappa_0, true_cov_matrix=None, expected_returns=None, previous_weights=None):
    """Calculate portfolio weights using DSHRP method and evaluate metrics."""
    # Step 1: Hierarchical Clustering
    corr_matrix = returns.corr()
    distance_matrix = np.sqrt(0.5 * (1 - corr_matrix))
    condensed_distance_matrix = squareform(distance_matrix)
    link = linkage(condensed_distance_matrix, method='complete')
    cluster_labels = fcluster(link, t, criterion='distance')

    # Step 2: Dynamic Shrinkage for Clusters
    shrunk_cov_matrices = {}
    for label in np.unique(cluster_labels):
        assets_in_cluster = returns.columns[cluster_labels == label]
        cluster_returns = returns[assets_in_cluster]

        lw = LedoitWolf()
        cov_matrix = lw.fit(cluster_returns).covariance_

        eigenvalues = np.linalg.eigvals(cov_matrix)
        condition_number = np.max(eigenvalues) / np.min(eigenvalues)
        shrinkage_factor = 1 / (1 + np.exp(-alpha * (condition_number - kappa_0)))

        shrunk_cov = (1 - shrinkage_factor) * cov_matrix + shrinkage_factor * np.eye(len(cov_matrix))
        shrunk_cov_matrices[label] = shrunk_cov

    # Step 3: Combine and Quasi-Diagonalize
    def pad_matrix(matrix, target_size):
        padded = np.zeros((target_size, target_size))
        padded[:matrix.shape[0], :matrix.shape[1]] = matrix
        return padded

    max_size = max(matrix.shape[0] for matrix in shrunk_cov_matrices.values())
    padded_cov_matrices = [pad_matrix(shrunk_cov_matrices[i], max_size) for i in shrunk_cov_matrices]

    combined_cov_matrix = np.block([
        [padded_cov_matrices[i] if i == j else np.zeros((max_size, max_size))
         for j in range(len(padded_cov_matrices))]
        for i in range(len(padded_cov_matrices))
    ])

    sorted_indices = np.argsort(cluster_labels)
    quasi_diag_cov = combined_cov_matrix[sorted_indices, :][:, sorted_indices]

    # Step 4: Recursive Bisection for Weights Calculation
    std_dev = np.sqrt(np.diag(quasi_diag_cov))
    inv_vol = np.zeros_like(std_dev)
    non_zero_std = std_dev > 0
    inv_vol[non_zero_std] = 1 / std_dev[non_zero_std]

    if inv_vol.sum() > 0:
        weights = inv_vol / inv_vol.sum()
    else:
        weights = np.zeros_like(inv_vol)

    # Step 5: Evaluate Performance
    metrics = evaluate_portfolio_metrics(weights, returns, true_cov_matrix, expected_returns, previous_weights)

    return weights, metrics


In [ ]:
def higher_order_dshrp_weights_sim(data, rf_rate, alpha, beta, baseline_volatility, baseline_correlation, gamma, delta,
                                   true_cov_matrix=None, expected_returns=None, previous_weights=None):
    """
    Calculate dynamically adjusted covariance matrices, optimize weights for Sharpe ratio,
    and evaluate metrics.
    """
    n_assets = data.shape[1]
    adjusted_covariances = []
    final_weights = None

    expected_returns = data.mean().values

    for t in range(len(data)):
        row_data = data.iloc[:t + 1]

        if len(row_data) < 2:
            continue

        centered_data = row_data - row_data.mean()
        cov_matrix = row_data.cov().values

        skewness_tensor = np.einsum('ti,tj,tk->ijk', centered_data.values, centered_data.values, centered_data.values) / len(row_data)
        kurtosis_tensor = np.einsum('ti,tj,tk,tl->ijkl', centered_data.values, centered_data.values, centered_data.values, centered_data.values) / len(row_data)
        kurtosis_tensor -= 3 * np.einsum('ij,kl->ijkl', cov_matrix, cov_matrix)

        initial_weights = np.ones(n_assets) / n_assets
        portfolio_variance = np.dot(initial_weights.T, np.dot(cov_matrix, initial_weights))
        portfolio_volatility = np.sqrt(portfolio_variance)
        correlation_matrix = row_data.corr().values
        upper_triangle_indices = np.triu_indices_from(correlation_matrix, k=1)
        avg_correlation = np.nanmean(correlation_matrix[upper_triangle_indices])

        lambda_dynamic = 1 / (1 + np.exp(-gamma * (portfolio_volatility - baseline_volatility) + delta * (avg_correlation - baseline_correlation)))
        nonlin_covariance = cov_matrix + alpha * np.sum(skewness_tensor, axis=2) + beta * np.sum(kurtosis_tensor, axis=(2, 3))
        adjusted_covariance = (1 - lambda_dynamic) * cov_matrix + lambda_dynamic * nonlin_covariance

        adjusted_covariance = (adjusted_covariance + adjusted_covariance.T) / 2
        eigvals, eigvecs = np.linalg.eigh(adjusted_covariance)
        eigvals = np.clip(eigvals, 1e-8, None)
        adjusted_covariance = eigvecs @ np.diag(eigvals) @ eigvecs.T

        std_dev = np.sqrt(np.diag(adjusted_covariance))
        inv_vol = np.zeros_like(std_dev)
        non_zero_std = std_dev > 0
        inv_vol[non_zero_std] = 1 / std_dev[non_zero_std]

        if inv_vol.sum() > 0:
            weights = inv_vol / inv_vol.sum()
        else:
            weights = np.zeros_like(inv_vol)

        final_weights = weights

    metrics = evaluate_portfolio_metrics(final_weights, data, true_cov_matrix, expected_returns, previous_weights)

    return final_weights, metrics


In [ ]:
def find_optimal_params_sim(returns, rf_rates, param_grid, true_cov_matrix, expected_returns):
    best_sr, best_params = -np.inf, None
    best_metrics = None

    for params in ParameterGrid(param_grid):
        t, alpha, kappa_0 = params['t'], params['alpha'], params['kappa_0']
        weights, metrics = dshrp_weights_sim(returns, rf_rates, t, alpha, kappa_0, true_cov_matrix, expected_returns)

        if metrics['Sharpe Ratio'] > best_sr:
            best_sr = metrics['Sharpe Ratio']
            best_params = params
            best_metrics = metrics

    return best_params, best_metrics


In [ ]:
def find_optimal_params_higher_order_sim(returns, rf_rate, param_grid, true_cov_matrix, expected_returns):
    best_sr, best_params = -np.inf, None
    best_metrics = None

    for params in ParameterGrid(param_grid):
        weights, metrics = higher_order_dshrp_weights_sim(returns, rf_rate, **params,
                                                      true_cov_matrix=true_cov_matrix, expected_returns=expected_returns)

        if metrics['Sharpe Ratio'] > best_sr:
            best_sr = metrics['Sharpe Ratio']
            best_params = params
            best_metrics = metrics

    return best_params, best_metrics


### Simulation Results

#### Generate Simulation Data

In [ ]:
# Reproducibility
np.random.seed(0)

# Parameters for simulation
N = 5  # Number of assets
T = 500  # Number of time periods

# Generate synthetic prices and returns
simulated_prices_ho, simulated_log_returns_ho = simulate_multivariate_garch_with_higher_moments(N, T)

# Display results
print(simulated_prices_ho.head())
print(simulated_log_returns_ho.head())

# Risk-Free Rate Simulation
rf_base_rate = 0.03  # Base rate of 3%
rf_volatility = 0.002  # Small volatility
simulated_rf_rate_ho = pd.Series(rf_base_rate + rf_volatility * np.random.randn(T), name="RiskFreeRate")

# Expected Returns (using sample mean)
expected_returns = simulated_log_returns_ho.mean().values

# True Covariance Matrix (using sample covariance)
true_cov_matrix = simulated_log_returns_ho.cov().values

       Asset_1     Asset_2     Asset_3     Asset_4     Asset_5
0   100.000000  100.000000  100.000000  100.000000  100.000000
1   271.828183   36.787944  271.828183  271.828183  216.036271
2   421.215149  100.000000  100.000000  100.000000  138.502036
3   441.995375  176.158366   36.787944   83.956172   75.194374
4  1201.467997   91.003972   31.243721   44.266159  204.399501
    Asset_1   Asset_2   Asset_3   Asset_4   Asset_5
0  0.000000  0.000000  0.000000  0.000000  0.000000
1  1.000000 -1.000000  1.000000  1.000000  0.770276
2  0.437974  1.000000 -1.000000 -1.000000 -0.444561
3  0.048156  0.566213 -1.000000 -0.174875 -0.610809
4  1.000000 -0.660480 -0.163352 -0.640074  1.000000


#### Apply DS-HRP Optimization

In [ ]:
# Find Optimal Parameters for DS-HRP
best_params_dshrp, best_metrics_dshrp = find_optimal_params_sim(
    simulated_log_returns_ho,
    simulated_rf_rate_ho,
    param_grid_dshrp,
    true_cov_matrix,
    expected_returns
)

print("\nBest Parameters for DS-HRP:")
print(best_params_dshrp)
print("Best Metrics for DS-HRP:")
print(best_metrics_dshrp)



Best Parameters for DS-HRP:
{'alpha': 0.1, 'kappa_0': 10, 't': 0.1}
Best Metrics for DS-HRP:
{'Out-of-Sample Variance': 0.024619021047527938, 'Sharpe Ratio': 0.0439189869337476, 'Maximum Drawdown': 0.9966831922806407, 'Portfolio Turnover': 0.006183669776134132}


#### Apply Higher-Order DS-HRP Optimization

In [ ]:
# Find Optimal Parameters for Higher-Order DS-HRP
best_params_ho_dshrp, best_metrics_ho_dshrp = find_optimal_params_higher_order_sim(
    simulated_log_returns_ho,
    simulated_rf_rate_ho,
    param_grid_hom_dshrp,
    true_cov_matrix,
    expected_returns
)

print("\nBest Parameters for Higher-Order DS-HRP:")
print(best_params_ho_dshrp)
print("Best Metrics for Higher-Order DS-HRP:")
print(best_metrics_ho_dshrp)



Best Parameters for Higher-Order DS-HRP:
{'alpha': 0.0, 'baseline_correlation': 0.1, 'baseline_volatility': 0.32999999999999996, 'beta': 2, 'delta': 20, 'gamma': 10}
Best Metrics for Higher-Order DS-HRP:
{'Out-of-Sample Variance': 0.07197425286516748, 'Sharpe Ratio': 0.05384002897651184, 'Maximum Drawdown': 0.9999995535151202, 'Portfolio Turnover': 0.2829789572221066}


#### Evaluate Performance

In [ ]:
# best_params_dshrp = {'alpha': 0.1, 'kappa_0': 10, 't': 0.1}
# best_params_ho_dshrp = {'alpha': 0.0, 'baseline_correlation': 0.1, 'baseline_volatility': 0.32999999999999996, 'beta': 2, 'delta': 20, 'gamma': 10}

In [ ]:
# Evaluate DS-HRP Performance
weights_dshrp, metrics_dshrp = dshrp_weights_sim(
    simulated_log_returns_ho,
    simulated_rf_rate_ho,
    **best_params_dshrp,
    true_cov_matrix=true_cov_matrix,
    expected_returns=expected_returns
)

print("\nDS-HRP Portfolio Evaluation:")
print(metrics_dshrp)

# Evaluate Higher-Order DS-HRP Performance
weights_ho_dshrp, metrics_ho_dshrp = higher_order_dshrp_weights_sim(
    simulated_log_returns_ho,
    simulated_rf_rate_ho,
    **best_params_ho_dshrp,
    true_cov_matrix=true_cov_matrix,
    expected_returns=expected_returns
)

print("\nHigher-Order DS-HRP Portfolio Evaluation:")
print(metrics_ho_dshrp)



DS-HRP Portfolio Evaluation:
{'Out-of-Sample Variance': 0.024619021047527938, 'Sharpe Ratio': 0.0439189869337476, 'Maximum Drawdown': 0.9966831922806407, 'Portfolio Turnover': 0.006183669776134132}

Higher-Order DS-HRP Portfolio Evaluation:
{'Out-of-Sample Variance': 0.07197425286516748, 'Sharpe Ratio': 0.05384002897651184, 'Maximum Drawdown': 0.9999995535151202, 'Portfolio Turnover': 0.2829789572221066}


#### Visualize Results

In [ ]:
# Create the figure
fig = go.Figure()

# Add DS-HRP Metrics
fig.add_trace(go.Bar(
    x=['Out-of-Sample Variance', 'Sharpe Ratio', 'Max Drawdown', 'Portfolio Turnover'],
    y=[
        metrics_dshrp['Out-of-Sample Variance'],
        metrics_dshrp['Sharpe Ratio'],
        metrics_dshrp['Maximum Drawdown'],
        metrics_dshrp['Portfolio Turnover']
    ],
    name='DS-HRP',
    marker_color='blue',
    offsetgroup=0  # Group for better spacing
))

# Add Higher-Order DS-HRP Metrics
fig.add_trace(go.Bar(
    x=['Out-of-Sample Variance', 'Sharpe Ratio', 'Max Drawdown', 'Portfolio Turnover'],
    y=[
        metrics_ho_dshrp['Out-of-Sample Variance'],
        metrics_ho_dshrp['Sharpe Ratio'],
        metrics_ho_dshrp['Maximum Drawdown'],
        metrics_ho_dshrp['Portfolio Turnover']
    ],
    name='HOM DS-HRP',
    marker_color='orange',
    offsetgroup=1  # Group for better spacing
))

# Update layout with requested customizations
fig.update_layout(
    title={
        'text': 'Performance Metrics Comparison: DS-HRP vs Higher-Order Moment DS-HRP',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Metrics',
    yaxis_title='Values',
    barmode='group',  # Ensures bars are side by side
    bargap=0.2,  # Adds spacing between groups of bars
    bargroupgap=0.1,  # Adds spacing between bars within a group
    legend=dict(
        x=0.5,
        xanchor='center',
        y=-0.15,
        orientation='h',
        font=dict(size=12)
    ),
    width=1000,  # Set custom width
    height=500,  # Set custom height
    margin=dict(l=10, r=10, t=50, b=0)  # Add margins for clean spacing
)

# Save the plot as an image
fig.write_image("/content/drive/My Drive/data/simulation_ho_performance_plotly.png", scale=3)

# Display the plot
fig.show()

**Out-of-Sample Variance:** The Out-of-Sample Variance is higher for HOM DS-HRP (0.072) compared to DS-HRP (0.025). This suggests that while HOM DS-HRP incorporates advanced covariance adjustments, it may introduce additional variability in the portfolio's returns.

**Sharpe Ratio:** The Sharpe Ratio is marginally higher for HOM DS-HRP (0.054) compared to DS-HRP (0.044). This indicates that HOM DS-HRP slightly improves risk-adjusted returns, validating the effectiveness of higher-order moment adjustments in capturing complex return dynamics.

**Maximum Drawdown:** Both frameworks experience substantial drawdowns, with HOM DS-HRP (100.00\%) and DS-HRP (99.67\%). This indicates that neither method fully mitigates extreme downside risks in highly volatile synthetic market conditions.

**Portfolio Turnover:** HOM DS-HRP shows significantly higher Portfolio Turnover (0.283) compared to DS-HRP (0.006). This suggests that HOM DS-HRP requires more frequent adjustments to maintain its optimized allocation, potentially increasing transaction costs in practical applications.

### Influence of Hyperparameters

In [ ]:
def evaluate_hyperparameters_dshrp(returns, rf_rate, param_ranges, true_cov_matrix, expected_returns):
    """
    Evaluate the influence of hyperparameters on DS-HRP and Higher-Order DS-HRP evaluation metrics.

    Parameters:
    - returns: DataFrame of asset returns.
    - rf_rate: Series of risk-free rates.
    - param_ranges: Dictionary of hyperparameter ranges.
    - true_cov_matrix: True covariance matrix of returns.
    - expected_returns: Expected returns vector.
    """
    results = {'DSHRP': {}, 'HODSHRP': {}}

    # Initialize results storage
    for model in results.keys():
        for param in param_ranges.keys():
            results[model][param] = {'SR': [], 'Vol': [], 'MDD': [], 'Turnover': []}

    ### Evaluate DS-HRP ###
    for param, values in param_ranges.items():
        for value in values:
            if param in ['dshrp_t', 'dshrp_alpha', 'dshrp_kappa_0']:
                weights, metrics = dshrp_weights_sim(
                    returns,
                    rf_rate,
                    t=value if param == 'dshrp_t' else 0.5,
                    alpha=value if param == 'dshrp_alpha' else 0.5,
                    kappa_0=value if param == 'dshrp_kappa_0' else 20,
                    true_cov_matrix=true_cov_matrix,
                    expected_returns=expected_returns
                )
                results['DSHRP'][param]['SR'].append(metrics['Sharpe Ratio'])
                results['DSHRP'][param]['Vol'].append(metrics['Out-of-Sample Variance'])
                results['DSHRP'][param]['MDD'].append(metrics['Maximum Drawdown'])
                results['DSHRP'][param]['Turnover'].append(metrics['Portfolio Turnover'])

    ### Evaluate Higher-Order DS-HRP ###
    for param, values in param_ranges.items():
        for value in values:
            if param in ['hom_dshrp_alpha', 'hom_dshrp_beta', 'hom_dshrp_sigma_0', 'hom_dshrp_rho_0', 'hom_dshrp_gamma', 'hom_dshrp_delta']:
                weights, metrics = higher_order_dshrp_weights_sim(
                    returns,
                    rf_rate,
                    alpha=value if param == 'hom_dshrp_alpha' else 0.5,
                    beta=value if param == 'hom_dshrp_beta' else 5,
                    baseline_volatility=value if param == 'hom_dshrp_sigma_0' else 0.1,
                    baseline_correlation=value if param == 'hom_dshrp_rho_0' else 0.3,
                    gamma=value if param == 'hom_dshrp_gamma' else 10,
                    delta=value if param == 'hom_dshrp_delta' else 10,
                    true_cov_matrix=true_cov_matrix,
                    expected_returns=expected_returns
                )
                results['HODSHRP'][param]['SR'].append(metrics['Sharpe Ratio'])
                results['HODSHRP'][param]['Vol'].append(metrics['Out-of-Sample Variance'])
                results['HODSHRP'][param]['MDD'].append(metrics['Maximum Drawdown'])
                results['HODSHRP'][param]['Turnover'].append(metrics['Portfolio Turnover'])

    ### Visualization using Plotly ###
    fig = make_subplots(
        rows=3,
        cols=3,
        subplot_titles=[
            "DSHRP: t", "DSHRP: alpha", "DSHRP: kappa_0",
            "HOM DSHRP: alpha", "HOM DSHRP: beta",
            "HOM DSHRP: baseline_volatility", "HOM DSHRP: baseline_correlation",
            "HOM DSHRP: gamma", "HOM DSHRP: delta"
        ],
        vertical_spacing=0.05,
        horizontal_spacing=0.05
    )

    # Define consistent colors for each metric
    metric_colors = {
        "Vol": "#1f77b4",        # Blue
        "SR": "#ff7f0e",         # Orange
        "MDD": "#2ca02c",        # Green
        "Turnover": "#d62728"    # Red
    }

    # Legend Labels
    legend_labels = {
        "SR": "Sharpe Ratio",
        "Vol": "Out-of-Sample Variance",
        "MDD": "Maximum Drawdown",
        "Turnover": "Portfolio Turnover"
    }

    # Plot Metrics for Each Parameter
    row, col = 1, 1
    for param, values in param_ranges.items():
        for metric in ["Vol", "SR", "MDD", "Turnover"]:
            color = metric_colors[metric]

            # DS-HRP Plot
            if param in ['dshrp_t', 'dshrp_alpha', 'dshrp_kappa_0']:
                fig.add_trace(
                    go.Scatter(
                        x=values,
                        y=results['DSHRP'][param][metric],
                        mode='lines+markers',
                        name=legend_labels[metric] if row == 1 and col == 1 else None,
                        marker=dict(symbol='circle', size=6),
                        line=dict(dash='solid', color=color),
                        showlegend=(row == 1 and col == 1)
                    ),
                    row=row,
                    col=col
                )

            # HOM DS-HRP Plot
            else:
                fig.add_trace(
                    go.Scatter(
                        x=values,
                        y=results['HODSHRP'][param][metric],
                        mode='lines+markers',
                        name=legend_labels[metric] if row == 1 and col == 1 else None,
                        marker=dict(symbol='square', size=6),
                        line=dict(dash='solid', color=color),
                        showlegend=(row == 1 and col == 1)
                    ),
                    row=row,
                    col=col
                )

        # Adjust rows and columns
        col += 1
        if col > 3:
            col = 1
            row += 1

    ### Update Layout ###
    fig.update_layout(
        title={
            'text': "Hyperparameter Sensitivity Analysis for DS-HRP and Higher-Order DS-HRP",
            'x': 0.5,
            'xanchor': 'center'
        },
        font=dict(size=12),
        height=1200,
        width=1500,
        margin=dict(l=10, r=10, t=60, b=10),
        legend=dict(
            x=0.5,
            xanchor='center',
            y=-0.03,
            orientation='h',
            font=dict(size=12)
        )
    )

    ### Save the Plot ###
    fig.write_image("/content/drive/My Drive/data/simulation_ho_hyperparameter_plotly.png", scale=3)

    ### Display the Plot ###
    fig.show()


param_ranges = {
    "dshrp_t": np.linspace(0, 2, 20),
    "dshrp_alpha": np.linspace(0, 2, 20),
    "dshrp_kappa_0": np.linspace(0, 50, 20),
    "hom_dshrp_alpha": np.linspace(0, 2, 20),
    "hom_dshrp_beta": np.linspace(0, 10, 20),
    "hom_dshrp_sigma_0": np.linspace(0, 1, 20),
    "hom_dshrp_rho_0": np.linspace(-1, 1, 20),
    "hom_dshrp_gamma": np.linspace(0, 20, 20),
    "hom_dshrp_delta": np.linspace(0, 20, 20)
}

evaluate_hyperparameters_dshrp(
    simulated_log_returns_ho,
    simulated_rf_rate_ho,
    param_ranges,
    true_cov_matrix,
    expected_returns
)


**For DS-HRP:**

- **Parameter $t$:** The sensitivity analysis reveals that the **Out-of-Sample Variance** and **Sharpe Ratio** are relatively stable across most values of $t$. However, **Maximum Drawdown** exhibits significant fluctuations at lower values of $t$, indicating that inappropriate calibration may result in heightened portfolio risk. **Portfolio Turnover** remains consistently low, suggesting minimal rebalancing activity.

- **Parameter $\alpha$:** Changes in $\alpha$ have limited effects on **Out-of-Sample Variance** and **Sharpe Ratio**. However, **Maximum Drawdown** shows slight sensitivity, and **Portfolio Turnover** remains relatively stable, indicating minimal adjustment activity across different values of $\alpha$.

- **Parameter $\kappa_0$:** As $\kappa_0$ increases, **Out-of-Sample Variance** slightly decreases, while **Maximum Drawdown** and **Sharpe Ratio** remain mostly unchanged. **Portfolio Turnover** is generally unaffected, suggesting this parameter primarily impacts covariance shrinkage with limited influence on allocation stability.

---

**For HOM DS-HRP:**

- **Parameter $\alpha$:** Higher values of $\alpha$ steadily increase **Maximum Drawdown** and marginally improve **Out-of-Sample Variance** and **Sharpe Ratio**. This suggests that while higher-order moment adjustments improve risk allocation, they may also increase drawdown sensitivity.

- **Parameter $\beta$:** Increasing $\beta$ shows a steep rise in **Maximum Drawdown**, peaking at moderate values before plateauing. **Out-of-Sample Variance** and **Sharpe Ratio** improve slightly, while **Portfolio Turnover** remains stable across lower values but increases under aggressive calibration.

- **Parameter $\sigma_0$:** As $\sigma_0$ approaches higher thresholds, **Maximum Drawdown** and **Out-of-Sample Variance** drop sharply, while **Sharpe Ratio** and **Portfolio Turnover** converge to minimal levels. This indicates that overly restrictive volatility thresholds may suppress portfolio responsiveness.

- **Parameter $\rho_0$:** The correlation threshold $\rho_0$ demonstrates a clear breakpoint: below a certain negative threshold, **Maximum Drawdown** spikes dramatically, while **Sharpe Ratio** and **Out-of-Sample Variance** exhibit minimal changes. Proper calibration is crucial to avoid instability.

- **Parameter $\gamma$:** Changes in $\gamma$ reveal moderate increases in **Maximum Drawdown**, while other metrics remain largely unaffected. This suggests diminishing returns from excessive sensitivity to volatility changes.

- **Parameter $\delta$:** Similar to $\gamma$, **Maximum Drawdown** increases with rising $\delta$ values, plateauing at higher levels. **Out-of-Sample Variance**, **Sharpe Ratio**, and **Portfolio Turnover** show minimal sensitivity, highlighting diminishing benefits from excessive correlation sensitivity.
